In [1]:
# Импорт библиотек
import re  # Для работы с регулярными выражениями
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Чтение датасета
df = pd.read_csv('data/data.csv', sep=';')

In [3]:
display(df)

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Заголовок документа,Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Дата рождения пациента
0,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Лейкоциты (WBC),1.80,10*9/л,4:11,Пониж,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
1,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Эритроциты (RBC),4.18,10*12/л,"3,8:5,7",Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
2,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Гемоглобин (HGB),120.00,г/л,117:160,Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
3,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Гематокрит (HCT),35.10,%,35:47,Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
4,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Средний объем эритроцита (MCV),84.00,фл,80:99,Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36292,516f652e-8bb8-11eb-bb95-2cea7fe73f75,Острый лейкоз неуточненного клеточного типа,Общий анализ крови,Базофилы,1.00,%,0:2,Норм,Комментарий,Тромбоциты проверены. Рекомендована консультац...,_,М,2014-10-17 00:00:00
36293,516f652e-8bb8-11eb-bb95-2cea7fe73f75,Острый лейкоз неуточненного клеточного типа,Общий анализ крови,Эозинофилы,0.00,%,0:5,Норм,Комментарий,Тромбоциты проверены. Рекомендована консультац...,_,М,2014-10-17 00:00:00
36294,516f652e-8bb8-11eb-bb95-2cea7fe73f75,Острый лейкоз неуточненного клеточного типа,Общий анализ крови,Пролимфоциты,0.00,%,0:0,Норм,Комментарий,Тромбоциты проверены. Рекомендована консультац...,_,М,2014-10-17 00:00:00
36295,516f652e-8bb8-11eb-bb95-2cea7fe73f75,Острый лейкоз неуточненного клеточного типа,Общий анализ крови,Лимфоциты,65.00,%,18:44,Повыш,Комментарий,Тромбоциты проверены. Рекомендована консультац...,_,М,2014-10-17 00:00:00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36297 entries, 0 to 36296
Data columns (total 13 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   ID истории болезни                    36297 non-null  object 
 1   Осн. диаг. при выписке МКБ10 (текст)  36297 non-null  object 
 2   Заголовок документа                   36297 non-null  object 
 3   Кол. лаб. показатель                  36236 non-null  object 
 4   Значение кол. показателя              36236 non-null  float64
 5   Ед. изм. кол. показателя              36236 non-null  object 
 6   Норма кол. показателя                 36236 non-null  object 
 7   Флаг нормы кол. показателя            36236 non-null  object 
 8   Кач. лаб. показатель                  7005 non-null   object 
 9   Значение кач. показателя              7005 non-null   object 
 10  Норма кач. показателя                 7005 non-null   object 
 11  Пол            

In [5]:
# Удаляем неинформативные столбцы (и так понятно что у нас документ -- ОАК)
print(f'Всего уникальных значений в Заголовке документа: {df["Заголовок документа"].nunique()}')
df = df.drop('Заголовок документа', axis=1)

Всего уникальных значений в Заголовке документа: 1


### Этап 1: Перекодировка пола в числовой признак

(перекодируем значение пола и добавим обработку неожиданных значений)

In [6]:
# Проверяем уникальные значения в столбце "Пол", чтобы убедиться в наличии только ожидаемых значений
unique_gender_values = df['Пол'].unique()
print(f"Уникальные значения в столбце 'Пол' перед преобразованием: {unique_gender_values}")

Уникальные значения в столбце 'Пол' перед преобразованием: ['Ж' 'М']


In [7]:
# Проверяем и обрабатываем значения
if set(unique_gender_values) <= {"Ж", "М"}:
    # Заменяем "Ж" на 0 и "М" на 1
    df['Пол'] = df['Пол'].map({"Ж": 0, "М": 1})
    print("Значения в столбце 'Пол' успешно заменены на числовые коды.")
else:
    # Если найдены другие значения, выводим предупреждение и не изменяем данные
    print("Предупреждение: найдены неожиданные значения в столбце 'Пол'. Проверьте данные перед продолжением.")

Значения в столбце 'Пол' успешно заменены на числовые коды.


### Этап 2: Расчет возраста пациентов



In [8]:
# Преобразование в формат даты с обработкой некорректных значений
df['Дата рождения пациента'] = pd.to_datetime(df['Дата рождения пациента'], errors='coerce')
print("Столбец 'Дата рождения пациента' успешно преобразован в формат даты.")

Столбец 'Дата рождения пациента' успешно преобразован в формат даты.


In [9]:
# Удаляем строки с некорректными датами рождения, если такие имеются
initial_row_count = df.shape[0]
df = df.dropna(subset=['Дата рождения пациента'])
dropped_rows = initial_row_count - df.shape[0]
print(f"Количество удаленных строк с некорректными датами рождения: {dropped_rows}")

Количество удаленных строк с некорректными датами рождения: 0


In [10]:
# Расчет возраста пациента
today = datetime.today()
df['Возраст'] = df['Дата рождения пациента'].apply(
    lambda birth_date: today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
)

In [11]:
# Удаление столбца "Дата рождения пациента"
df = df.drop(columns=['Дата рождения пациента'])
print("Столбец 'Дата рождения пациента' удален из данных после расчета возраста.")

Столбец 'Дата рождения пациента' удален из данных после расчета возраста.


### Этап 3: Обработка пропусков в столбце "Кол. лаб. показатель"



In [12]:
# Выборка с пропущенными значениями по столбцу Кол. лаб. показатель
df_na = df[df["Кол. лаб. показатель"].isna()]
print(f'Всего строк с пропусками по показателям крови: {df_na.shape[0]}')
print(f'Комментарии: {df_na["Кач. лаб. показатель"].unique()}')
print(f'Причины в комментариях: {df_na["Значение кач. показателя"].unique()}')

Всего строк с пропусками по показателям крови: 61
Комментарии: ['Комментарий']
Причины в комментариях: ['сгусток крови!' 'Сгусток' 'сгусток крови'
 'тромбоциты по мазку 81х10*9/л' ' тромбоциты по мазку 23*10*9/л'
 'тромбоциты по мазку 21' 'формулу подсчитать невозможно'
 'тромбоциты пртоверены' 'встречаются микросгустки тромбоцитов' 'сгусток'
 'тромбоциты проверены по мазку'
 'шизоциты 4:1000 эритроцитов(№до 10:1000) ' 'нет на месте'
 'тромбоциты при подсчете по мазку 102' ',' 'ТРОМЮОЦИТЫ ПРОВЕРЕНЫ'
 'тромбоциты проверены' 'Тромбоциты проверены.' 'сгусток в крови'
 'тромбоциты по мазку 64*10*9/Л' 'Тромбоциты проверены\t' 'тромб.пров'
 'СОЭ - 7 мм/ч'
 'Лейкоциты после пересчета (без учета нормобластов) - 14,4х10х9/л. Тромбоциты проверены.  Моноцитоидные элементы отличаются полимрфизмом ядер,  наличием зернистости в цитоплазме , высоким ядерноцитоплазматическим соотношением .'
 'подсчитано 50 клеток, моноцитоидные элементы -16'
 'моноцитарные элементы с нуклеолами в ядре 63,5'
 'Тромбоци

In [13]:
# Определяем значения, при которых строки должны быть удалены
values_to_delete = [
    'сгусток крови!', 'Сгусток', 'сгусток крови', 'формулу подсчитать невозможно',
    'тромбоциты пртоверены', 'встречаются микросгустки тромбоцитов', 'сгусток',
    'тромбоциты проверены по мазку', 'нет на месте', 'сгусток в крови', ',',
    'ТРОМЮОЦИТЫ ПРОВЕРЕНЫ', 'тромбоциты проверены', 'Тромбоциты проверены.', 'Тромбоциты проверены\t', 'тромб.пров',
    'Тромбоциты проверены. Большинство бластных клеток представляет собой клетки крупных и средних размеров, с высоким ядерно-цитоплазматическим соотношением, уродливыми ядрами (неправильной формы ядра). хроматин сглаженный, встречаются нуклеолы. Цитоплазма у части клеток вакуолизированная.',
    'лейкоцитарную формулу подсчитать невозможно', 'подсчет л/ф невозможен',
    'моноцидоидные элементы с грубой структурой хроматина и остатаками нуклеол',
    'Тромбоциты проверены. Подсчет лейкоцитарной формулы невозможен из-за крайне низкого количества лейкоцитов крови.'
]

In [14]:
# Удаляем строки с определенными значениями в "Значение кач. показателя"
initial_row_count = df.shape[0]
df = df[~(df['Кол. лаб. показатель'].isna() & df['Значение кач. показателя'].isin(values_to_delete))]
deleted_rows = initial_row_count - df.shape[0]
print(f"Количество удаленных строк: {deleted_rows}")

Количество удаленных строк: 44


### Этап 4: Объединение одинаковых показателей

Цель: Объединить одинаковые показатели, которые представлены разными наименованиями и единицами, чтобы сделать данные более унифицированными и удобными для анализа.

In [15]:
# Вывод всех уникальных значений в колонке "Кол. лаб. показатель"
unique_lab_indicators = df['Кол. лаб. показатель'].unique()
print("Уникальные значения в колонке 'Кол. лаб. показатель':")
print(unique_lab_indicators)

Уникальные значения в колонке 'Кол. лаб. показатель':
['Лейкоциты (WBC)' 'Эритроциты (RBC)' 'Гемоглобин (HGB)'
 'Гематокрит (HCT)' 'Средний объем эритроцита (MCV)'
 'Среднее содержание гемоглобина в эритроците (MCH)'
 'Средняя концентрация гемоглобина в эритроците (MCHC)' 'Тромбоциты (PLT)'
 'Нейтрофилы, относительное количество (NE%)'
 'Лимфоциты, относительное количество (LY%)'
 'Нейтрофилы, абсолютное количество (NE#)'
 'Лимфоциты, абсолютное количество (LY#)'
 'Смешанная фракция, абсолютное количество (MXD#)'
 'Смешанная фракция, относительное количество (MXD%)'
 'Ширина распределения эритроцитов по объему, коэффициент вариации (RDW-CV)'
 'Ширина распределения эритроцитов по объему, стандартное отклонение (RDW-SD)'
 'Ширина распределения эритроцитов по объему (RDW)' 'Тромбокрит (PCT)'
 'Средний объем тромбоцита (MPV)' 'Ширина распределения тромбоцитов (PDW)'
 'Моноциты, относительное количество (MO%)'
 'Эозинофилы, относительное количество (EO%)'
 'Базофилы, относительное количеств

In [16]:
# Создаем словарь для унификации показателей и приведения их к общему названию
unification_dict = {
    'Лейкоциты (WBC)': 'Лейкоциты (WBC)',
    'WBC': 'Лейкоциты (WBC)',
    'Общее количество лейкоцитов (WBC)': 'Лейкоциты (WBC)',
    'Лейкоциты': 'Лейкоциты (WBC)', # проверено
    
    'Эритроциты (RBC)': 'Эритроциты (RBC)',
    'RBC': 'Эритроциты (RBC)',
    'Общее количество эритроцитов (RBC)': 'Эритроциты (RBC)',
    'Эритроциты': 'Эритроциты (RBC)', # проверено
    
    'Гемоглобин (HGB)': 'Гемоглобин (HGB)',
    'HGB': 'Гемоглобин (HGB)',
    'Гемоглобин': 'Гемоглобин (HGB)', # проверено
    
    'Гематокрит (HCT)': 'Гематокрит (HCT)',
    'HCT': 'Гематокрит (HCT)',
    'Гематокрит': 'Гематокрит (HCT)', # проверено

    'Тромбоциты (PLT)': 'Тромбоциты (PLT)',
    'PLT': 'Тромбоциты (PLT)',
    'Тромбоциты': 'Тромбоциты (PLT)', # проверено
    
    'Средний объем эритроцита (MCV)': 'Средний объем эритроцита (MCV)',
    'MCV': 'Средний объем эритроцита (MCV)',
    'Средний объем эритроцита': 'Средний объем эритроцита (MCV)', # проверено
    
    'Среднее содержание гемоглобина в эритроците (MCH)': 'Среднее содержание гемоглобина в эритроците (MCH)',
    'MCH': 'Среднее содержание гемоглобина в эритроците (MCH)',
    'Среднее содержание гемоглобина в эритроците': 'Среднее содержание гемоглобина в эритроците (MCH)', # проверено
    
    'Средняя концентрация гемоглобина в эритроците (MCHC)': 'Средняя концентрация гемоглобина в эритроците (MCHC)',
    'MCHC': 'Средняя концентрация гемоглобина в эритроците (MCHC)',
    'Средняя концентрация гемоглобина в эритроците': 'Средняя концентрация гемоглобина в эритроците (MCHC)', # проверено

    'Средний объем тромбоцита (MPV)': 'Средний объем тромбоцита (MPV)',
    'MPV': 'Средний объем тромбоцита (MPV)',
    'MPV ': 'Средний объем тромбоцита (MPV)',
    'MPV (Средний объём тромбоцитов)': 'Средний объем тромбоцита (MPV)',
    'Средний объём тромбоцитов': 'Средний объем тромбоцита (MPV)',
    'Средний объём тромбоцита': 'Средний объем тромбоцита (MPV)', # проверено

    'Тромбокрит (PCT)': 'Тромбокрит (PCT)',
    'PCT': 'Тромбокрит (PCT)',
    'Тромбокрит': 'Тромбокрит (PCT)', # проверено
    
    'Ширина распределения эритроцитов по объему (RDW)': 'Распределение эритроцитов по объему (RDW)',
    'Ширина распределения эритроцитов': 'Распределение эритроцитов по объему (RDW)',
    'RDW': 'Распределение эритроцитов по объему (RDW)', # проверено
    'Ширина распределения эритроцитов (RDW)': 'Распределение эритроцитов по объему (RDW)',

    'Ширина распределения эритроцитов по объему, коэффициент вариации (RDW-CV)': 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)',
    'Ширина распределения эритроцитов по объему, коэффициент вариации': 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)',
    'RDW-CV': 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)',
    'RDW-CV ': 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)', # проверено

    'Ширина распределения эритроцитов по объему, стандартное отклонение (RDW-SD)': 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)',
    'Ширина распределения эритроцитов по объему, стандартное отклонение': 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)', 
    'Ширина распределения эритроцитов, стандартное отклонение (RDW-SD)': 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)',
    'RDW-SD': 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)', # проверено

    'Ширина распределения тромбоцитов (PDW)': 'Распределение тромбоцитов по объему (PDW)',
    'Ширина распределения тромбоцитов': 'Распределение тромбоцитов по объему (PDW)',
    'Ширина распределения тромбоцитов по объему': 'Распределение тромбоцитов по объему (PDW)',
    'PDW': 'Распределение тромбоцитов по объему (PDW)', # проверено

    'Скорость оседания эритроцитов (СОЭ) по Вестергрену': 'СОЭ по Вестергрену',
    'СОЭ Вест.': 'СОЭ по Вестергрену',
    # 'СОЭ по Панч.': 'СОЭ', я предлагаю вообще дропнуть эти показатели, т.к.
    # метод Вестергрена одобрен в качестве эталонного Международным комитетом по стандартизации в гематологии (ICSH) 
    # и регулируется стандартом Института клинических и лабораторных стандартов (CLSI), а по Панченкову
    # вроде как считается устаревшим...
    'СОЭ': 'СОЭ по Вестергрену', # проверено
    
    'Нейтрофилы, относительное количество (NE%)': 'Нейтрофилы, относительное количество (NE%)',
    'Относительное количество нейтрофилов (NE%)': 'Нейтрофилы, относительное количество (NE%)',
    'NE%': 'Нейтрофилы, относительное количество (NE%)',
    'Нейтрофилы %': 'Нейтрофилы, относительное количество (NE%)',
    'Нейтрофилы, относительное количество': 'Нейтрофилы, относительное количество (NE%)', # проверено
    
    'Нейтрофилы, абсолютное количество (NE#)': 'Нейтрофилы, абсолютное количество (NE#)',
    'Абсолютное количество нейтрофилов (NE#)': 'Нейтрофилы, абсолютное количество (NE#)',
    'Нейтрофилы #': 'Нейтрофилы, абсолютное количество (NE#)',
    'Нейтрофилы, абсолютное количество': 'Нейтрофилы, абсолютное количество (NE#)', # проверено
    
    'Лимфоциты, относительное количество (LY%)': 'Лимфоциты, относительное количество (LY%)',
    'Относительное количество лимфоцитов (LY%)': 'Лимфоциты, относительное количество (LY%)',
    'LY%': 'Лимфоциты, относительное количество (LY%)',
    'Лимфоциты %': 'Лимфоциты, относительное количество (LY%)',
    'Лимфоциты, относительное количество': 'Лимфоциты, относительное количество (LY%)', # проверено
    
    'Лимфоциты, абсолютное количество (LY#)': 'Лимфоциты, абсолютное количество (LY#)',
    'Абсолютное количество лимфоцитов (LY#)': 'Лимфоциты, абсолютное количество (LY#)',
    'Лимфоциты #': 'Лимфоциты, абсолютное количество (LY#)',
    'Лимфоциты, абсолютное количество': 'Лимфоциты, абсолютное количество (LY#)', # проверено

    'Моноциты, относительное количество (MO%)': 'Моноциты, относительное количество (MO%)',
    'Моноциты, относительное количество': 'Моноциты, относительное количество (MO%)',
    'Относительное количество моноцитов (MO%)': 'Моноциты, относительное количество (MO%)',
    'MO%': 'Моноциты, относительное количество (MO%)',
    'Моноциты %': 'Моноциты, относительное количество (MO%)', # проверено

    'Моноциты, абсолютное количество (MO#)': 'Моноциты, абсолютное количество (MO#)',
    'Моноциты, абсолютное количество': 'Моноциты, абсолютное количество (MO#)',
    'Абсолютное количество моноцитов (MO#)': 'Моноциты, абсолютное количество (MO#)',
    'Моноциты #': 'Моноциты, абсолютное количество (MO#)', # проверено

    'Эозинофилы, относительное количество (EO%)': 'Эозинофилы, относительное количество (EO%)',
    'Эозинофилы, относительное количество': 'Эозинофилы, относительное количество (EO%)',
    'Относительное количество эозинофилов (EO%)': 'Эозинофилы, относительное количество (EO%)',
    'Эозинофилы %': 'Эозинофилы, относительное количество (EO%)',
    'EO%': 'Эозинофилы, относительное количество (EO%)', # проверено

    'Эозинофилы, абсолютное количество (EO#)': 'Эозинофилы, абсолютное количество (EO#)',
    'Эозинофилы, абсолютное количество': 'Эозинофилы, абсолютное количество (EO#)',
    'Абсолютное количество эозинофилов (EO#)': 'Эозинофилы, абсолютное количество (EO#)',
    'Эозинофилы # ': 'Эозинофилы, абсолютное количество (EO#)',
    'Эозинофилы #': 'Эозинофилы, абсолютное количество (EO#)', # проверено

    'Базофилы, относительное количество (BA%)': 'Базофилы, относительное количество (BA%)',
    'Базофилы, относительное количество': 'Базофилы, относительное количество (BA%)',
    'Относительное количество базофилов (BA%)': 'Базофилы, относительное количество (BA%)',
    'Базофилы %': 'Базофилы, относительное количество (BA%)',
    'BA%': 'Базофилы, относительное количество (BA%)', # проверено

    'Базофилы, абсолютное количество (BA#)': 'Базофилы, абсолютное количество (BA#)',
    'Абсолютное количество базофилов (BA#)': 'Базофилы, абсолютное количество (BA#)',
    'Базофилы, абсолютное количество': 'Базофилы, абсолютное количество (BA#)',
    'Базофилы #': 'Базофилы, абсолютное количество (BA#)',
    'Базофилы # ': 'Базофилы, абсолютное количество (BA#)', # проверено
    
    'Смешанная фракция, относительное количество (MXD%)': 'Смешанная фракция, относительное количество (MXD%)',
    'Смешанная фракция, относительное количество': 'Смешанная фракция, относительное количество (MXD%)',
    'MXD%': 'Смешанная фракция, относительное количество (MXD%)', # проверено
        
    'Смешанная фракция, абсолютное количество (MXD#)': 'Смешанная фракция, абсолютное количество (MXD#)',
    'Смешанная фракция, абсолютное количество': 'Смешанная фракция, абсолютное количество (MXD#)',
    'MXD#': 'Смешанная фракция, абсолютное количество (MXD#)',
    'MXD# ': 'Смешанная фракция, абсолютное количество (MXD#)', # проверено

    'Нормобласты %': 'Нормобласты, относительное количество (%)', # проверено
    'Нормобласты #': 'Нормобласты, абсолютное количество (#)', # проверено
    'Ретикулоциты %': 'Ретикулоциты, абсолютное количество (#)', # проверено
    'Ретикулоциты кол-во': 'Ретикулоциты, абсолютное количество (#)', # проверено
    'RET%': 'Ретикулоциты, относительное количество (%)', # проверено
    'Цветовой показатель': 'Цветовой показатель', 
    
    'Палочкоядерные': 'Палочкоядерные нейтрофилы (%)',
    'Сегментоядерные': 'Сегментоядерные нейтрофилы (%)',
    'Базофилы': 'Базофилы (лейкоцитарная формула, %)',
    'Лимфоциты': 'Лимфоциты (лейкоцитарная формула, %)',
    'Моноциты': 'Моноциты (лейкоцитарная формула, %)',
    'Эозинофилы': 'Эозинофилы (лейкоцитарная формула, %)',
    'Плазматические клетки': 'Плазматические клетки',
    'Плазматич. клетки': 'Плазматические клетки',
    'Миелоциты': 'Миелоциты (лейкоцитарная формула, %)',
    'Промиелоциты': 'Промиелоциты (лейкоцитарная формула, %)',
    'Пролимфоциты': 'Пролимфоциты (лейкоцитарная формула, %)',
    'Нормобласты': 'Нормобласты',
    'P-LCR': 'P-LCR',
}


In [17]:
# Применяем унификацию названий
df['Кол. лаб. показатель'] = df['Кол. лаб. показатель'].replace(unification_dict)
print("Наименования показателей успешно унифицированы.")

Наименования показателей успешно унифицированы.


In [18]:
new_unique_lab_indicators = df['Кол. лаб. показатель'].unique()
print("Новые уникальные значения в колонке 'Кол. лаб. показатель':")
print(new_unique_lab_indicators)

Новые уникальные значения в колонке 'Кол. лаб. показатель':
['Лейкоциты (WBC)' 'Эритроциты (RBC)' 'Гемоглобин (HGB)'
 'Гематокрит (HCT)' 'Средний объем эритроцита (MCV)'
 'Среднее содержание гемоглобина в эритроците (MCH)'
 'Средняя концентрация гемоглобина в эритроците (MCHC)' 'Тромбоциты (PLT)'
 'Нейтрофилы, относительное количество (NE%)'
 'Лимфоциты, относительное количество (LY%)'
 'Нейтрофилы, абсолютное количество (NE#)'
 'Лимфоциты, абсолютное количество (LY#)'
 'Смешанная фракция, абсолютное количество (MXD#)'
 'Смешанная фракция, относительное количество (MXD%)'
 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)'
 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)'
 'Распределение эритроцитов по объему (RDW)' 'Тромбокрит (PCT)'
 'Средний объем тромбоцита (MPV)'
 'Распределение тромбоцитов по объему (PDW)'
 'Моноциты, относительное количество (MO%)'
 'Эозинофилы, относительное количество (EO%)'
 'Базофилы, относительное количество (BA%)'
 '

In [19]:
# Список значений для удаления
red_highlighted_values = [
    'Средний объем тромбоцита', 'Юные', 'Бласты', 'Нормобласты', 
    'СОЭ Панч.', 'Неклассифицируемые кол-во',
    'Неклассифицируемые %', 'MXD', 'СОЭ по Панченкову'
]

In [20]:
# Фильтруем строки с показателями, которые выделены красным цветом
filtered_df = df[df['Кол. лаб. показатель'].isin(red_highlighted_values)]

In [21]:
# Считаем количество каждого показателя
count_per_indicator = filtered_df['Кол. лаб. показатель'].value_counts()

# Выводим результаты
print("Количество записей по каждому показателю, выделенному красным цветом:")
print(count_per_indicator)

Количество записей по каждому показателю, выделенному красным цветом:
Кол. лаб. показатель
Юные                         304
Неклассифицируемые %         239
СОЭ Панч.                    225
Нормобласты                  174
Бласты                       143
Неклассифицируемые кол-во    117
MXD                           49
Средний объем тромбоцита       1
СОЭ по Панченкову              1
Name: count, dtype: int64


короче дропну я начиная с юных, мне сложно их интерпертировать, а результат может пострадать.... если что отменить можно всегда

In [22]:
# Список значений, которые нужно удалить из столбца "Кол. лаб. показатель"
values_to_remove = [
    'Юные', 'Неклассифицируемые %', 'СОЭ Панч.', 'Нормобласты', 
    'Бласты', 'Неклассифицируемые кол-во', 'MXD', 'Средний объем тромбоцита', 
    'СОЭ по Панченкову'
]

In [23]:
# Удаление строк, где "Кол. лаб. показатель" имеет одно из значений в списке
df = df[~df['Кол. лаб. показатель'].isin(values_to_remove)]

In [24]:
# Проверка результата
print("Строки с указанными значениями удалены.")
print(df['Кол. лаб. показатель'].value_counts())

Строки с указанными значениями удалены.
Кол. лаб. показатель
Гемоглобин (HGB)                                                        1619
Гематокрит (HCT)                                                        1619
Средний объем эритроцита (MCV)                                          1619
Среднее содержание гемоглобина в эритроците (MCH)                       1619
Средняя концентрация гемоглобина в эритроците (MCHC)                    1619
Эритроциты (RBC)                                                        1619
Тромбоциты (PLT)                                                        1617
Лейкоциты (WBC)                                                         1616
Лимфоциты, относительное количество (LY%)                               1470
Лимфоциты, абсолютное количество (LY#)                                  1470
Нейтрофилы, относительное количество (NE%)                              1332
Нейтрофилы, абсолютное количество (NE#)                                 1319
Средний объем т

Думаю удалить также следующие показали по причине того, что их мало в нашем датасете и они мало информативны:
* P-LCR  
* Плазматические клетки 

In [25]:
# Список значений, которые нужно удалить из столбца "Кол. лаб. показатель"
values_to_remove2 = [
    'P-LCR', 'Плазматические клетки',
]

In [26]:
# Удаление строк, где "Кол. лаб. показатель" имеет одно из значений в списке
df = df[~df['Кол. лаб. показатель'].isin(values_to_remove2)]

In [27]:
# Проверка результата
print("Строки с указанными значениями удалены.")
print(df['Кол. лаб. показатель'].value_counts())

Строки с указанными значениями удалены.
Кол. лаб. показатель
Гемоглобин (HGB)                                                        1619
Гематокрит (HCT)                                                        1619
Средний объем эритроцита (MCV)                                          1619
Среднее содержание гемоглобина в эритроците (MCH)                       1619
Средняя концентрация гемоглобина в эритроците (MCHC)                    1619
Эритроциты (RBC)                                                        1619
Тромбоциты (PLT)                                                        1617
Лейкоциты (WBC)                                                         1616
Лимфоциты, абсолютное количество (LY#)                                  1470
Лимфоциты, относительное количество (LY%)                               1470
Нейтрофилы, относительное количество (NE%)                              1332
Нейтрофилы, абсолютное количество (NE#)                                 1319
Средний объем т

## Что дальше: 

1.  нужно обработать колонки:

   `Ед. изм. кол. показателя` (привести все единицы измерения к стандартным), --- Done
   
   `Норма кол. показателя` (проверить нормы, привести к стандартным), --- почти Done (нужно проверить за мной все и особено подумать над АТЕНШНТ моментами)	
   
   `Флаг нормы кол. показателя` (тут тоже привести к "Норм", "Пониж", "Повыш"),	-- Done, но нужно отработать пропуски
   
   `Кач. лаб. показатель` (разобраться, можно ли вообще дропнуть этот столбец?),
   
   `Значение кач. показателя` (тут остались комментарии, которые содержат полезную инфу для столбца "Значение кол. показателя", нужно ее оттуда достать и поставить на место) ,	
   
   `Норма кач. показателя` (разобраться, можно ли вообще дропнуть этот столбец?) 

In [28]:
print(df['Ед. изм. кол. показателя'].value_counts())

Ед. изм. кол. показателя
%          14270
10*9/л      8808
фл          3665
г/л         2528
10*12/л     1619
пг          1619
_            885
г/дл         715
мм/час       568
fL           136
FL            59
Name: count, dtype: int64


In [29]:
# Фильтрация строк, где "Ед. изм. кол. показателя" равно "г/дл"
gdl_df = df[df['Ед. изм. кол. показателя'] == 'г/дл']

In [30]:
# Подсчет количества для каждого значения "г/дл" в "Кол. лаб. показатель"
count_per_lab = gdl_df['Кол. лаб. показатель'].value_counts()
count_per_lab

Кол. лаб. показатель
Средняя концентрация гемоглобина в эритроците (MCHC)    715
Name: count, dtype: int64

In [31]:
# Фильтрация строк, где "Ед. изм. кол. показателя" равно "г/л"
gl_df = df[df['Ед. изм. кол. показателя'] == 'г/л']

In [32]:
# Подсчет количества для каждого значения "г/л" в "Кол. лаб. показатель"
count_per_lab = gl_df['Кол. лаб. показатель'].value_counts()
count_per_lab

Кол. лаб. показатель
Гемоглобин (HGB)                                        1619
Средняя концентрация гемоглобина в эритроците (MCHC)     904
Распределение тромбоцитов по объему (PDW)                  5
Name: count, dtype: int64

нужно перевести г/дл в г/л, т.к. в интернете написано, что (MCHC) тоже измеряется в г/л

In [33]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Средняя концентрация гемоглобина в эритроците (MCHC)'
# и 'Ед. изм. кол. показателя' равен 'г/дл'
mask_hb = (df['Кол. лаб. показатель'] == 'Средняя концентрация гемоглобина в эритроците (MCHC)') & (df['Ед. изм. кол. показателя'] == 'г/дл')

# Умножаем значения в столбце 'Значение кол. показателя' на 10 для выбранных строк
df.loc[mask_hb, 'Значение кол. показателя'] *= 10

# Обновляем единицу измерения на 'г/л' для выбранных строк
df.loc[mask_hb, 'Ед. изм. кол. показателя'] = 'г/л'

print("Единицы измерения для показателя 'Средняя концентрация гемоглобина в эритроците (MCHC)' успешно преобразованы в г/л.")


Единицы измерения для показателя 'Средняя концентрация гемоглобина в эритроците (MCHC)' успешно преобразованы в г/л.


In [34]:
print(df['Ед. изм. кол. показателя'].value_counts())

Ед. изм. кол. показателя
%          14270
10*9/л      8808
фл          3665
г/л         3243
10*12/л     1619
пг          1619
_            885
мм/час       568
fL           136
FL            59
Name: count, dtype: int64


In [35]:
# Исправляем обозначения единиц для других показателей
# Пример: приводим FL и фл к одному обозначению
df['Ед. изм. кол. показателя'] = df['Ед. изм. кол. показателя'].replace({'FL': 'фл', 'fL': 'фл', 'фЛ': 'фл'})
print("Единицы измерения успешно унифицированы для обозначения 'фл'.")

Единицы измерения успешно унифицированы для обозначения 'фл'.


In [36]:
print(df['Ед. изм. кол. показателя'].value_counts())

Ед. изм. кол. показателя
%          14270
10*9/л      8808
фл          3860
г/л         3243
10*12/л     1619
пг          1619
_            885
мм/час       568
Name: count, dtype: int64


--------------------------------------------------------------------------------------------------------------
`Норма кол. показателя` (проверить нормы, привести к стандартным),	

In [37]:
print(df['Норма кол. показателя'].value_counts())

Норма кол. показателя
18:44          2135
45:72          2011
80:99          1604
150:450        1540
27:38          1530
               ... 
Общая норма       1
6,5:10,0          1
35:48             1
17:63,8           1
0,59:2,07         1
Name: count, Length: 116, dtype: int64


In [38]:
chvet_df = df[df['Кол. лаб. показатель'] == 'Цветовой показатель']

In [39]:
unique_un = chvet_df['Норма кол. показателя'].unique()

In [40]:
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Цветовой показатель':")
print(unique_un)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Цветовой показатель':
['0,8:1,05' '0,85:1,15' '_' '0,8:1,5' '0,80:1,05' '0.80:1.05' '0.8:1.05']


In [41]:
# Заменяем значения в столбце "Норма кол. показателя" на "0,85:1,05" для строк, где "Кол. лаб. показатель" равен "Цветовой показатель"
df.loc[df['Кол. лаб. показатель'] == 'Цветовой показатель', 'Норма кол. показателя'] = '0,85:1,00'

In [42]:
# Проверка результата
print("Измененные значения в столбце 'Норма кол. показателя' для 'Цветовой показатель':")
print(df[df['Кол. лаб. показатель'] == 'Цветовой показатель']['Норма кол. показателя'].unique())

Измененные значения в столбце 'Норма кол. показателя' для 'Цветовой показатель':
['0,85:1,00']


In [43]:
chvet_df_new = df[df['Кол. лаб. показатель'] == 'Цветовой показатель']

In [44]:
unique_un_new = chvet_df_new['Норма кол. показателя'].unique()

In [45]:
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Цветовой показатель':")
print(unique_un_new)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Цветовой показатель':
['0,85:1,00']


In [46]:
HGB_df = df[df['Кол. лаб. показатель'] == 'Гемоглобин (HGB)']

In [47]:
unique_HGB = HGB_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Гемоглобин (HGB)':")
print(unique_HGB)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Гемоглобин (HGB)':
['117:160' '132:180' '117:180' '117:140' '114:154' '132:160' '122:168']


In [48]:
# Норма взята с сайта: https://gemotest.ru/info/spravochnik/analizy/gemoglobin/
def get_hgb_norm(age, gender):
    """Возвращает норму гемоглобина в зависимости от возраста и пола."""
    if age < 18:
        # Норма для детей
        if age == 0:  # Возраст < 1 месяца (младенцы)
            return "152:235"
        elif 1 <= age <= 13 / 365:  # 2–13 дней
            return "150:240"
        elif 14 / 365 <= age <= 23 / 365:  # 14–23 дня
            return "127:187"
        elif 24 / 365 <= age <= 30 / 365:  # 24–30 дней
            return "103:179"
        elif age < 2 / 12:  # 1 месяц
            return "90:166"
        elif age < 3 / 12:  # 2 месяца
            return "92:150"
        elif age < 4 / 12:  # 3 месяца
            return "96:135"
        elif age < 5 / 12:  # 4 месяца
            return "96:135"
        elif age < 8 / 12:  # 5–7 месяцев
            return "101:132"
        elif age < 11 / 12:  # 8–10 месяцев
            return "105:135"
        elif age < 1:  # 11 месяцев
            return "107:131"
        elif age < 5:  # 12 месяцев — 4 года
            return "108:132"
        elif age < 10:  # 5–9 лет
            return "111:143"
        elif age < 12:  # 10–11 лет
            return "119:147"
        elif age < 15:  # 12–14 лет
            if gender == 1:  # Мальчики
                return "120:160"
            else:  # Девочки
                return "115:150"
        elif age < 18:  # 15–17 лет
            if gender == 1:  # Юноши
                return "117:166"
            else:  # Девушки
                return "117:153"
    else:
        # Норма для взрослых
        if gender == 1:  # Мужчины
            if 18 <= age <= 44:
                return "132:173"
            elif 45 <= age <= 64:
                return "131:172"
            else:  # От 65 лет
                return "126:174"
        else:  # Женщины
            if 18 <= age <= 44:
                return "117:155"
            elif 45 <= age <= 64:
                return "117:160"
            else:  # От 65 лет
                return "117:161"

In [49]:
# Применяем функцию для строк, где 'Кол. лаб. показатель' равно 'Гемоглобин (HGB)'
mask_hgb = df['Кол. лаб. показатель'] == 'Гемоглобин (HGB)'
df.loc[mask_hgb, 'Норма кол. показателя'] = df[mask_hgb].apply(lambda row: get_hgb_norm(row['Возраст'], row['Пол']), axis=1)

In [50]:
# Проверка результата
print("Установлены нормы гемоглобина для показателя 'Гемоглобин (HGB)'.")
print(df[mask_hgb][['Возраст', 'Пол', 'Норма кол. показателя']])

Установлены нормы гемоглобина для показателя 'Гемоглобин (HGB)'.
       Возраст  Пол Норма кол. показателя
2           62    0               117:160
18          62    0               117:160
48          62    0               117:160
78          62    0               117:160
106         62    0               117:160
...        ...  ...                   ...
36170       42    1               132:173
36197       42    1               132:173
36219       42    1               132:173
36246       42    1               132:173
36268       10    1               119:147

[1619 rows x 3 columns]


In [51]:
HCT_df = df[df['Кол. лаб. показатель'] == 'Гематокрит (HCT)']

In [52]:
unique_HCT = HCT_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Гематокрит (HCT)':")
print(unique_HCT)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Гематокрит (HCT)':
['35:47' '39:52' '35:52' '34:46' '35,0:47,0' '30.0:49.5' '39,0:52,0']


In [53]:
unique_ed_HCT = HCT_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Гематокрит (HCT)':")
print(unique_ed_HCT)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Гематокрит (HCT)':
['%']


In [54]:
# норма взята с сайта: https://gemotest.ru/info/spravochnik/analizy/gematokrit/
def get_hct_norm(age, gender):
    """Возвращает норму гематокрита в зависимости от возраста и пола."""
    if age < 18:
        # Норма для детей
        if age == 0:  # Возраст < 1 месяца (младенцы)
            return "41:65" if age <= 13 / 365 else "33:55"
        elif age < 2 / 12:  # 2 месяца
            return "32:44"
        elif age < 6 / 12:  # 3–5 месяцев
            return "31:41"
        elif age < 9 / 12:  # 6–8 месяцев
            return "32:40"
        elif age < 12 / 12:  # 9–11 месяцев
            return "33:41"
        elif age < 3:  # 12 месяцев — 2 года
            return "32:40"
        elif age < 6:  # 3–5 лет
            return "32:42"
        elif age < 9:  # 6–8 лет
            return "33:41"
        elif age < 12:  # 9–11 лет
            return "34:43"
        else:
            # Норма для подростков
            if gender == 1:  # Мужчины
                if 12 <= age < 15:
                    return "35:45"
                elif 15 <= age < 18:
                    return "37:48"
            else:  # Женщины
                if 12 <= age < 18:
                    return "34:44"
    else:
        # Норма для взрослых
        if gender == 1:  # Мужчины
            if 18 <= age <= 44:
                return "39:49"
            elif 45 <= age <= 64:
                return "39:50"
            else:  # От 65 лет
                return "37:51"
        else:  # Женщины
            if 18 <= age <= 44:
                return "35:45"
            else:  # От 45 лет
                return "35:47"

In [55]:
# Применяем функцию для строк, где 'Кол. лаб. показатель' равно 'Гематокрит (HCT)'
mask_hct = df['Кол. лаб. показатель'] == 'Гематокрит (HCT)'
df.loc[mask_hct, 'Норма кол. показателя'] = df[mask_hct].apply(lambda row: get_hct_norm(row['Возраст'], row['Пол']), axis=1)

In [56]:
# Проверка результата
print("Установлены нормы гематокрита для показателя 'Гематокрит (HCT)'.")
print(df[mask_hct][['Возраст', 'Пол', 'Норма кол. показателя']])

Установлены нормы гематокрита для показателя 'Гематокрит (HCT)'.
       Возраст  Пол Норма кол. показателя
3           62    0                 35:47
19          62    0                 35:47
49          62    0                 35:47
79          62    0                 35:47
107         62    0                 35:47
...        ...  ...                   ...
36171       42    1                 39:49
36198       42    1                 39:49
36220       42    1                 39:49
36247       42    1                 39:49
36269       10    1                 34:43

[1619 rows x 3 columns]


In [57]:
# норма взята с сайта: https://www.cmd-online.ru/analizy-i-tseny/katalog-analizov/sredniy-obem-eritrotsita/#:~:text=MCV%20%D1%8F%D0%B2%D0%BB%D1%8F%D0%B5%D1%82%D1%81%D1%8F%20%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%B8%D0%B2%D0%BD%D1%8B%D0%BC%20%D0%BF%D0%BE%D0%BA%D0%B0%D0%B7%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%BC%20%D0%B4%D0%B8%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D0%B0,%2C%20%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%B5%20100%20%D1%84%D0%BB%20%2D%20%D0%BC%D0%B0%D0%BA%D1%80%D0%BE%D1%86%D0%B8%D1%82%D0%BE%D0%B7.
def get_mcv_norm(age, gender):
    """Возвращает норму среднего объема эритроцита (MCV) в зависимости от возраста и пола."""
    if age < 2 / 52:  # < 2 недели
        return "88:140"
    elif age < 1 / 12:  # 2 недели - 1 месяц
        return "91:112"
    elif age < 2 / 12:  # 1 - 2 месяца
        return "84:106"
    elif age < 4 / 12:  # 2 - 4 месяца
        return "76:97"
    elif age < 6 / 12:  # 4 - 6 месяцев
        return "68:85"
    elif age < 9 / 12:  # 6 - 9 месяцев
        return "70:85"
    elif age < 2:  # 9 месяцев - 2 года
        return "71:84"
    elif age < 5:  # 2 года - 5 лет
        return "73:85"
    elif age < 9:  # 5 - 9 лет
        return "75:87"
    elif age < 12:  # 9 - 12 лет
        return "76:90"
    elif age < 15:  # 12 - 15 лет
        if gender == 1:  # Мужчины
            return "77:94"
        else:  # Женщины
            return "73:95"
    elif age < 18:  # 15 - 18 лет
        if gender == 1:  # Мужчины
            return "79:95"
        else:  # Женщины
            return "78:98"
    else:
        # Норма для взрослых
        if gender == 1:  # Мужчины
            if 18 <= age <= 45:
                return "80:99"
            elif 45 <= age <= 65:
                return "81:101"
            else:  # > 65 лет
                return "81:103"
        else:  # Женщины
            if 18 <= age <= 45:
                return "81:100"
            elif 45 <= age <= 65:
                return "81:101"
            else:  # > 65 лет
                return "81:102"

In [58]:
# Применяем функцию для строк, где 'Кол. лаб. показатель' равно 'Средний объем эритроцита (MCV)'
mask_mcv = df['Кол. лаб. показатель'] == 'Средний объем эритроцита (MCV)'
df.loc[mask_mcv, 'Норма кол. показателя'] = df[mask_mcv].apply(lambda row: get_mcv_norm(row['Возраст'], row['Пол']), axis=1)

# Проверка результата
print("Установлены нормы среднего объема эритроцита (MCV) для показателя 'Средний объем эритроцита (MCV)'.")
print(df[mask_mcv][['Возраст', 'Пол', 'Норма кол. показателя']])

Установлены нормы среднего объема эритроцита (MCV) для показателя 'Средний объем эритроцита (MCV)'.
       Возраст  Пол Норма кол. показателя
4           62    0                81:101
20          62    0                81:101
50          62    0                81:101
80          62    0                81:101
108         62    0                81:101
...        ...  ...                   ...
36172       42    1                 80:99
36199       42    1                 80:99
36221       42    1                 80:99
36248       42    1                 80:99
36270       10    1                 76:90

[1619 rows x 3 columns]


In [59]:
# https://www.cmd-online.ru/analizy-i-tseny/katalog-analizov/srednee-soderzhanie-gemoglobina-v-eritrotsite-mch-mean-corpuscular-hemoglobin/
def get_mch_norm(age, gender):
    """Возвращает норму среднего содержания гемоглобина в эритроците (MCH) в зависимости от возраста и пола."""
    if age < 2 / 52:  # < 2 недели
        return "30:37"
    elif age < 1 / 12:  # 2 недели - 1 месяц
        return "29:36"
    elif age < 2 / 12:  # 1 - 2 месяца
        return "27:34"
    elif age < 4 / 12:  # 2 - 4 месяца
        return "25:32"
    elif age < 6 / 12:  # 4 - 6 месяцев
        return "24:30"
    elif age < 9 / 12:  # 6 - 9 месяцев
        return "25:30"
    elif age < 1:  # 9 месяцев - 1 год
        return "24:30"
    elif age < 3:  # 1 - 3 года
        return "22:30"
    elif age < 9:  # 3 - 9 лет
        return "25:31"
    elif age < 15:  # 9 - 15 лет
        return "26:32"
    elif age < 18:  # 15 - 18 лет
        if gender == 1:  # Мужчины
            return "27:32"
        else:  # Женщины
            return "26:34"
    else:
        # Норма для взрослых
        if gender == 1:  # Мужчины
            if 18 <= age <= 45:
                return "27:34"
            elif 45 <= age <= 65:
                return "27:35"
            else:  # > 65 лет
                return "27:34"
        else:  # Женщины
            if 18 <= age <= 45:
                return "27:34"
            elif 45 <= age <= 65:
                return "27:34"
            else:  # > 65 лет
                return "27:35"

In [60]:
# Применяем функцию для строк, где 'Кол. лаб. показатель' равно 'Среднее содержание гемоглобина в эритроците (MCH)'
mask_mch = df['Кол. лаб. показатель'] == 'Среднее содержание гемоглобина в эритроците (MCH)'
df.loc[mask_mch, 'Норма кол. показателя'] = df[mask_mch].apply(lambda row: get_mch_norm(row['Возраст'], row['Пол']), axis=1)

# Проверка результата
print("Установлены нормы среднего содержания гемоглобина в эритроците (MCH) для показателя 'Среднее содержание гемоглобина в эритроците (MCH)'.")
print(df[mask_mch][['Возраст', 'Пол', 'Норма кол. показателя']])

Установлены нормы среднего содержания гемоглобина в эритроците (MCH) для показателя 'Среднее содержание гемоглобина в эритроците (MCH)'.
       Возраст  Пол Норма кол. показателя
5           62    0                 27:34
21          62    0                 27:34
51          62    0                 27:34
81          62    0                 27:34
109         62    0                 27:34
...        ...  ...                   ...
36173       42    1                 27:34
36200       42    1                 27:34
36222       42    1                 27:34
36249       42    1                 27:34
36271       10    1                 26:32

[1619 rows x 3 columns]


In [61]:
# https://gemotest.ru/info/spravochnik/analizy/eritrotsitarnye-indeksy/#:~:text=%D0%A1%D1%80%D0%B5%D0%B4%D0%BD%D1%8F%D1%8F%20%D0%BA%D0%BE%D0%BD%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D1%86%D0%B8%D1%8F%20%D0%B3%D0%B5%D0%BC%D0%BE%D0%B3%D0%BB%D0%BE%D0%B1%D0%B8%D0%BD%D0%B0%20%D0%B2%20%D1%8D%D1%80%D0%B8%D1%82%D1%80%D0%BE%D1%86%D0%B8%D1%82%D0%B5%20(%D0%9C%D0%A1%D0%9D%D0%A1)&text=%D0%9D%D0%B0%D0%B8%D0%B1%D0%BE%D0%BB%D0%B5%D0%B5%20%D0%B2%D1%8B%D1%81%D0%BE%D0%BA%D0%B8%D0%B5%20%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8F%20%E2%80%94%20%D1%83%20%D0%BD%D0%BE%D0%B2%D0%BE%D1%80%D0%BE%D0%B6%D0%B4%D1%91%D0%BD%D0%BD%D1%8B%D1%85,%D0%BF%D0%BE%D1%81%D1%82%D0%B5%D0%BF%D0%B5%D0%BD%D0%BD%D0%BE%20%D1%81%D0%BD%D0%B8%D0%B6%D0%B0%D1%8E%D1%82%D1%81%D1%8F%20%D0%B4%D0%BE%20%D0%BD%D0%BE%D1%80%D0%BC%D1%8B%20%D0%B2%D0%B7%D1%80%D0%BE%D1%81%D0%BB%D1%8B%D1%85.&text=%D0%A3%20%D0%B2%D0%B7%D1%80%D0%BE%D1%81%D0%BB%D1%8B%D1%85%20(%D0%BC%D1%83%D0%B6%D1%87%D0%B8%D0%BD%20%D0%B8%20%D0%B6%D0%B5%D0%BD%D1%89%D0%B8%D0%BD,300%E2%80%93380%20%D0%B3%2F%D0%BB.
def get_mchc_norm(age, gender):
    """Возвращает норму средней концентрации гемоглобина в эритроците (MCHC) в зависимости от возраста и пола."""
    if age < 1 / 12:  # 1 день - 1 месяц
        return "316:375"
    elif age < 5 / 12:  # 2 - 5 месяцев
        return "306:324"
    elif age < 7 / 12:  # 6 - 7 месяцев
        return "307:324"
    elif age < 1:  # 8 месяцев - 1 год
        return "297:324"
    elif age < 3:  # 2 года
        return "307:344"
    elif age < 10:  # 3 - 9 лет
        return "336:344"
    elif age < 15:  # 10 - 14 лет
        return "336:354"
    elif age < 18:  # 15 - 18 лет
        return "300:380"
    else:
        # Норма для взрослых, независимо от пола
        return "300:380"

In [62]:
# Применяем функцию для строк, где 'Кол. лаб. показатель' равно 'Средняя концентрация гемоглобина в эритроците (MCHC)'
mask_mchc = df['Кол. лаб. показатель'] == 'Средняя концентрация гемоглобина в эритроците (MCHC)'
df.loc[mask_mchc, 'Норма кол. показателя'] = df[mask_mchc].apply(lambda row: get_mchc_norm(row['Возраст'], row['Пол']), axis=1)

# Проверка результата
print("Установлены нормы средней концентрации гемоглобина в эритроците (MCHC) для показателя 'Средняя концентрация гемоглобина в эритроците (MCHC)'.")
print(df[mask_mchc][['Возраст', 'Пол', 'Норма кол. показателя']])

Установлены нормы средней концентрации гемоглобина в эритроците (MCHC) для показателя 'Средняя концентрация гемоглобина в эритроците (MCHC)'.
       Возраст  Пол Норма кол. показателя
6           62    0               300:380
22          62    0               300:380
52          62    0               300:380
82          62    0               300:380
110         62    0               300:380
...        ...  ...                   ...
36174       42    1               300:380
36201       42    1               300:380
36223       42    1               300:380
36250       42    1               300:380
36272       10    1               336:354

[1619 rows x 3 columns]


In [63]:
RBC_df = df[df['Кол. лаб. показатель'] == 'Эритроциты (RBC)']

In [64]:
unique_RBC = RBC_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Эритроциты (RBC)':")
print(unique_RBC)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Эритроциты (RBC)':
['3,8:5,7' '4,3:6,1' '4,30:6,10' '3,8:6,1' '3.5:5.2' '3,5:5,2' '3,80:5,70'
 '3.8:5,7' '3.1:5.7']


In [65]:
unique_ed_RBC = RBC_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Эритроциты (RBC)':")
print(unique_ed_RBC)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Эритроциты (RBC)':
['10*12/л']


In [66]:
# https://gemotest.ru/info/spravochnik/analizy/eritrotsity-krasnye-krovyanye-teltsa/
def get_rbc_norm(age, gender):
    """Возвращает норму эритроцитов (RBC) в зависимости от возраста и пола."""
    if age < 1 / 12:  # 1–13 дней
        return "3,9:5,9"
    elif age < 1:  # 14–30 дней, до 1 года
        if age < 2 / 12:
            return "3,3:5,3"
        elif age < 3 / 12:
            return "3,5:5,1"
        elif age < 4 / 12:
            return "3,6:4,8"
        elif age < 5 / 12:
            return "3,8:4,6"
        elif age < 6 / 12:
            return "4,0:4,8"
        elif age < 10 / 12:
            return "3,8:4,6"
        else:
            return "3,9:4,7"
    elif age < 5:  # 1 - 5 лет
        return "4,0:4,4"
    elif age == 6:
        return "4,1:4,5"
    elif age == 7:
        return "4,0:4,4"
    elif age == 8:
        return "4,2:4,6"
    elif age == 9:
        return "4,1:4,5"
    elif age < 12:  # 10 - 11 лет
        return "4,2:4,6"
    elif age < 15:  # 12–14 лет
        if gender == 1:  # Мальчики
            return "4,1:5,2"
        else:  # Девочки
            return "3,8:5,0"
    elif age < 18:  # 15–18 лет
        if gender == 1:  # Юноши
            return "4,2:5,6"
        else:  # Девушки
            return "3,9:5,1"
    else:
        # Норма для взрослых
        if gender == 1:  # Мужчины
            if 18 <= age <= 44:
                return "4,3:5,7"
            elif 45 <= age <= 64:
                return "4,2:5,6"
            else:  # > 65 лет
                return "3,8:5,8"
        else:  # Женщины
            if 18 <= age <= 44:
                return "3,8:5,1"
            elif 45 <= age <= 64:
                return "3,8:5,3"
            else:  # > 65 лет
                return "3,8:5,2"

In [67]:
# Применяем функцию для строк, где 'Кол. лаб. показатель' равно 'Эритроциты (RBC)'
mask_rbc = df['Кол. лаб. показатель'] == 'Эритроциты (RBC)'
df.loc[mask_rbc, 'Норма кол. показателя'] = df[mask_rbc].apply(lambda row: get_rbc_norm(row['Возраст'], row['Пол']), axis=1)

# Проверка результата
print("Установлены нормы эритроцитов (RBC) для показателя 'Эритроциты (RBC)'.")
print(df[mask_rbc][['Возраст', 'Пол', 'Норма кол. показателя']])

Установлены нормы эритроцитов (RBC) для показателя 'Эритроциты (RBC)'.
       Возраст  Пол Норма кол. показателя
1           62    0               3,8:5,3
17          62    0               3,8:5,3
47          62    0               3,8:5,3
77          62    0               3,8:5,3
105         62    0               3,8:5,3
...        ...  ...                   ...
36169       42    1               4,3:5,7
36196       42    1               4,3:5,7
36218       42    1               4,3:5,7
36245       42    1               4,3:5,7
36267       10    1               4,2:4,6

[1619 rows x 3 columns]


In [68]:
# https://gemotest.ru/info/spravochnik/analizy/trombotsity/#:~:text=%D0%B2%20%D0%B8%D0%BC%D0%BC%D1%83%D0%BD%D0%BD%D0%BE%D0%BC%20%D0%BE%D1%82%D0%B2%D0%B5%D1%82%D0%B5.-,%D0%9D%D0%BE%D1%80%D0%BC%D0%B0%20%D1%82%D1%80%D0%BE%D0%BC%D0%B1%D0%BE%D1%86%D0%B8%D1%82%D0%BE%D0%B2%20%D0%B2%20%D0%BA%D1%80%D0%BE%D0%B2%D0%B8%20%D1%83%20%D0%BC%D1%83%D0%B6%D1%87%D0%B8%D0%BD%20%D0%B8%20%D0%B6%D0%B5%D0%BD%D1%89%D0%B8%D0%BD,9%20%D0%BA%D0%BB%D0%B5%D1%82%D0%BE%D0%BA%20%D0%BD%D0%B0%20%D0%BB%D0%B8%D1%82%D1%80%20%D0%BA%D1%80%D0%BE%D0%B2%D0%B8.
def get_plt_norm(age):
    """Возвращает норму тромбоцитов (PLT) в зависимости от возраста."""
    # Нормы для детей до 1 года
    if age < 1 / 12:
        return "208:410"
    elif age < 2 / 12:
        return "208:352"
    elif age < 3 / 12:
        return "207:373"
    elif age < 4 / 12:
        return "205:395"
    elif age < 5 / 12:
        return "205:375"
    elif age < 6 / 12:
        return "203:377"
    elif age < 7 / 12:
        return "206:374"
    elif age < 8 / 12:
        return "215:365"
    elif age < 9 / 12:
        return "199:361"
    elif age < 10 / 12:
        return "205:355"
    elif age < 11 / 12:
        return "203:357"
    elif age < 1:
        return "207:353"
    
    # Нормы для детей старше 1 года
    elif age < 2:
        return "218:362"
    elif age < 3:
        return "214:366"
    elif age < 4:
        return "209:351"
    elif age < 5:
        return "196:344"
    elif age < 6:
        return "208:332"
    elif age < 7:
        return "220:360"
    elif age < 8:
        return "205:355"
    elif age < 9:
        return "205:375"
    elif age < 10:
        return "217:343"
    elif age < 11:
        return "211:349"
    elif age < 12:
        return "198:342"
    elif age < 13:
        return "202:338"
    elif age < 14:
        return "192:328"
    elif age < 15:
        return "198:342"
    elif age < 16:
        return "200:360"
    elif age < 17:
        return "180:320"
    
    # Норма для взрослых (с 18 лет и старше)
    return "180:320"

In [69]:
# Применяем функцию для строк, где 'Кол. лаб. показатель' равно 'Тромбоциты (PLT)'
mask_plt = df['Кол. лаб. показатель'] == 'Тромбоциты (PLT)'
df.loc[mask_plt, 'Норма кол. показателя'] = df[mask_plt].apply(lambda row: get_plt_norm(row['Возраст']), axis=1)

# Проверка результата
print("Установлены нормы тромбоцитов (PLT) для показателя 'Тромбоциты (PLT)'.")
print(df[mask_plt][['Возраст', 'Пол', 'Норма кол. показателя']])

Установлены нормы тромбоцитов (PLT) для показателя 'Тромбоциты (PLT)'.
       Возраст  Пол Норма кол. показателя
7           62    0               180:320
24          62    0               180:320
54          62    0               180:320
84          62    0               180:320
112         62    0               180:320
...        ...  ...                   ...
36176       42    1               180:320
36202       42    1               180:320
36225       42    1               180:320
36251       42    1               180:320
36273       10    1               211:349

[1617 rows x 3 columns]


In [70]:
# https://gemotest.ru/info/spravochnik/analizy/obshchiy-analiz-krovi/
def get_wbc_norm(age, gender):
    """Возвращает норму лейкоцитов (WBC) в зависимости от возраста и пола."""
    # Нормы для детей
    if age < 10 / 12:
        return "6,0:17,5"
    elif age < 8:
        return "5,5:15,5"
    elif age < 12:
        return "4,5:13,5"
    elif age < 15:
        return "4,5:13,0"
    elif age < 18:
        return "4,5:11,3"
    
    # Нормы для мужчин
    if gender == 1:  # Мужчины
        if 12 <= age <= 16:
            return "4,5:13,0"
        else:
            return "4,5:11,3"
    
    # Нормы для женщин
    else:  # Женщины
        if 12 <= age <= 16:
            return "4,5:13,0"
        else:
            return "4,5:11,3"

In [71]:
# Применяем функцию для строк, где 'Кол. лаб. показатель' равно 'Лейкоциты (WBC)'
mask_wbc = df['Кол. лаб. показатель'] == 'Лейкоциты (WBC)'
df.loc[mask_wbc, 'Норма кол. показателя'] = df[mask_wbc].apply(lambda row: get_wbc_norm(row['Возраст'], row['Пол']), axis=1)

# Проверка результата
print("Установлены нормы лейкоцитов (WBC) для показателя 'Лейкоциты (WBC)'.")
print(df[mask_wbc][['Возраст', 'Пол', 'Норма кол. показателя']])

Установлены нормы лейкоцитов (WBC) для показателя 'Лейкоциты (WBC)'.
       Возраст  Пол Норма кол. показателя
0           62    0              4,5:11,3
16          62    0              4,5:11,3
46          62    0              4,5:11,3
76          62    0              4,5:11,3
104         62    0              4,5:11,3
...        ...  ...                   ...
36168       42    1              4,5:11,3
36195       42    1              4,5:11,3
36217       42    1              4,5:11,3
36244       42    1              4,5:11,3
36266       10    1              4,5:13,5

[1616 rows x 3 columns]


In [72]:
PCT_df = df[df['Кол. лаб. показатель'] == 'Тромбокрит (PCT)']

In [73]:
unique_PCT = PCT_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Тромбокрит (PCT)':")
print(unique_PCT)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Тромбокрит (PCT)':
['0,19:0,36' '0,14:0,28']


In [74]:
unique_ed_PCT = PCT_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Тромбокрит (PCT)':")
print(unique_ed_PCT)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Тромбокрит (PCT)':
['%']


In [75]:
# https://medlineservice.ru/articles/trombokrit/#:~:text=%D0%9D%D0%BE%D1%80%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5%20%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8F%20%D1%82%D1%80%D0%BE%D0%BC%D0%B1%D0%BE%D0%BA%D1%80%D0%B8%D1%82%D0%B0,%D0%B2%D0%B7%D1%80%D0%BE%D1%81%D0%BB%D1%8B%D1%85%20%D0%B6%D0%B5%D0%BD%D1%89%D0%B8%D0%BD%2C%20%D0%BC%D1%83%D0%B6%D1%87%D0%B8%D0%BD%20%D0%B8%20%D0%B4%D0%B5%D1%82%D0%B5%D0%B9.
def get_pct_norm(age):
    """Возвращает норму тромбокрита (PCT) в зависимости от возраста."""
    # Нормы для детей до 18 лет
    if age < 18:
        return "0,15:0,35"
    # Норма для взрослых
    else:
        return "0,15:0,4"

In [76]:
# Применяем функцию для строк, где 'Кол. лаб. показатель' равно 'Тромбокрит (PCT)'
mask_pct = df['Кол. лаб. показатель'] == 'Тромбокрит (PCT)'
df.loc[mask_pct, 'Норма кол. показателя'] = df[mask_pct].apply(lambda row: get_pct_norm(row['Возраст']), axis=1)

# Проверка результата
print("Установлены нормы тромбокрита (PCT) для показателя 'Тромбокрит (PCT)'.")
print(df[mask_pct][['Возраст', 'Пол', 'Норма кол. показателя']])

Установлены нормы тромбокрита (PCT) для показателя 'Тромбокрит (PCT)'.
       Возраст  Пол Норма кол. показателя
25          62    0              0,15:0,4
55          62    0              0,15:0,4
85          62    0              0,15:0,4
113         62    0              0,15:0,4
144         62    0              0,15:0,4
...        ...  ...                   ...
36154       42    1              0,15:0,4
36177       42    1              0,15:0,4
36203       42    1              0,15:0,4
36226       42    1              0,15:0,4
36252       42    1              0,15:0,4

[925 rows x 3 columns]


In [77]:
MPV_df = df[df['Кол. лаб. показатель'] == 'Средний объем тромбоцита (MPV)']

In [78]:
unique_MPV = MPV_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Средний объем тромбоцита (MPV)':")
print(unique_MPV)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Средний объем тромбоцита (MPV)':
['9,1:12,6' '6,5:10' '6,5:12,6' '7.5:10.0' '7,5:10' '8,7:9,6' '6,5:10,0'
 '_']


In [79]:
unique_ed_MPV = MPV_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Средний объем тромбоцита (MPV)':")
print(unique_ed_MPV)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Средний объем тромбоцита (MPV)':
['фл' '_']


In [80]:
# Фильтруем строки, где 'Кол. лаб. показатель' равен 'Средний объем тромбоцита (MPV)'
mpv_mask = df['Кол. лаб. показатель'] == 'Средний объем тромбоцита (MPV)'

# Считаем количество '_' в столбце 'Норма кол. показателя' для фильтрованных строк
norma_underscore_count = df[mpv_mask]['Норма кол. показателя'].value_counts().get('_', 0)

# Считаем количество '_' в столбце 'Ед. изм. кол. показателя' для фильтрованных строк
unit_underscore_count = df[mpv_mask]['Ед. изм. кол. показателя'].value_counts().get('_', 0)

# Выводим результаты
print(f"Количество значений '_' в столбце 'Норма кол. показателя' для 'Средний объем тромбоцита (MPV)': {norma_underscore_count}")
print(f"Количество значений '_' в столбце 'Ед. изм. кол. показателя' для 'Средний объем тромбоцита (MPV)': {unit_underscore_count}")

Количество значений '_' в столбце 'Норма кол. показателя' для 'Средний объем тромбоцита (MPV)': 9
Количество значений '_' в столбце 'Ед. изм. кол. показателя' для 'Средний объем тромбоцита (MPV)': 5


In [81]:
mpv_norma_underscore_df = df[(df['Кол. лаб. показатель'] == 'Средний объем тромбоцита (MPV)') & (df['Норма кол. показателя'] == '_')]
mpv_norma_underscore_df

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Возраст
9836,f8b6efc7-215c-11ee-ab69-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),9.1,фл,_,Норм,Анизоцитоз,+,+-,1,70
9837,f8b6efc7-215c-11ee-ab69-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),9.1,фл,_,Норм,Пойкилоцитоз,++,+-,1,70
9838,f8b6efc7-215c-11ee-ab69-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),9.1,фл,_,Норм,Комментарий,"тромбоциты по мазку периферичекой крови 62х10*9,л",_,1,70
9970,f8b6efc7-215c-11ee-ab69-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),10.6,фл,_,Норм,Комментарий,тромбоциты проверены по мазку,_,1,70
14895,a6cc0de1-ac03-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),10.0,_,_,Норм,NaN,NaN,NaN,1,66
14980,a6cc0de1-ac03-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),9.7,_,_,Норм,NaN,NaN,NaN,1,66
16109,e150616f-b636-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),11.2,_,_,Норм,NaN,NaN,NaN,0,39
17750,7b9025cb-ae7c-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),10.6,_,_,Норм,NaN,NaN,NaN,0,72
17779,7b9025cb-ae7c-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),10.8,_,_,Норм,NaN,NaN,NaN,0,72


In [82]:
mpv_ed_underscore_df = df[(df['Кол. лаб. показатель'] == 'Средний объем тромбоцита (MPV)') & (df['Ед. изм. кол. показателя'] == '_')]
mpv_ed_underscore_df

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Возраст
14895,a6cc0de1-ac03-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),10.0,_,_,Норм,NaN,NaN,NaN,1,66
14980,a6cc0de1-ac03-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),9.7,_,_,Норм,NaN,NaN,NaN,1,66
16109,e150616f-b636-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),11.2,_,_,Норм,NaN,NaN,NaN,0,39
17750,7b9025cb-ae7c-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),10.6,_,_,Норм,NaN,NaN,NaN,0,72
17779,7b9025cb-ae7c-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Средний объем тромбоцита (MPV),10.8,_,_,Норм,NaN,NaN,NaN,0,72


делаем вывод что просто надо заменить '_' на 'фл'

In [83]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Средний объем тромбоцита (MPV)' и 'Ед. изм. кол. показателя' равно '_'
mpv_mask = (df['Кол. лаб. показатель'] == 'Средний объем тромбоцита (MPV)') & (df['Ед. изм. кол. показателя'] == '_')

# Заменяем значения '_' на 'фл' в отфильтрованных строках
df.loc[mpv_mask, 'Ед. изм. кол. показателя'] = 'фл'

In [84]:
new_MPV_df = df[df['Кол. лаб. показатель'] == 'Средний объем тромбоцита (MPV)']

In [85]:
new_unique_ed_MPV = new_MPV_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Средний объем тромбоцита (MPV)':")
print(new_unique_ed_MPV)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Средний объем тромбоцита (MPV)':
['фл']


In [86]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Средний объем тромбоцита (MPV)'
mpv_mask = df['Кол. лаб. показатель'] == 'Средний объем тромбоцита (MPV)'

# Задаем норму '7,8:11,0' в отфильтрованных строках в столбце 'Норма кол. показателя'
df.loc[mpv_mask, 'Норма кол. показателя'] = '7,8:11,0'

In [87]:
# Проверка изменений
print("Изменения в столбце 'Норма кол. показателя' для 'Средний объем тромбоцита (MPV)':")
print(df[mpv_mask][['Кол. лаб. показатель', 'Норма кол. показателя']])

Изменения в столбце 'Норма кол. показателя' для 'Средний объем тромбоцита (MPV)':
                 Кол. лаб. показатель Норма кол. показателя
26     Средний объем тромбоцита (MPV)              7,8:11,0
56     Средний объем тромбоцита (MPV)              7,8:11,0
86     Средний объем тромбоцита (MPV)              7,8:11,0
114    Средний объем тромбоцита (MPV)              7,8:11,0
145    Средний объем тромбоцита (MPV)              7,8:11,0
...                               ...                   ...
36165  Средний объем тромбоцита (MPV)              7,8:11,0
36178  Средний объем тромбоцита (MPV)              7,8:11,0
36214  Средний объем тромбоцита (MPV)              7,8:11,0
36227  Средний объем тромбоцита (MPV)              7,8:11,0
36263  Средний объем тромбоцита (MPV)              7,8:11,0

[981 rows x 2 columns]


In [88]:
RDW_df = df[df['Кол. лаб. показатель'] == 'Распределение эритроцитов по объему (RDW)']

In [89]:
unique_RDW = RDW_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Распределение эритроцитов по объему (RDW)':")
print(unique_RDW)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Распределение эритроцитов по объему (RDW)':
['10,5:18' '10:18' '10,5:18,0' '11:16' '11:18']


In [90]:
unique_ed_RDW = RDW_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Распределение эритроцитов по объему (RDW)':")
print(unique_ed_RDW)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Распределение эритроцитов по объему (RDW)':
['%']


In [91]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Распределение эритроцитов по объему (RDW)'
rdw_mask = df['Кол. лаб. показатель'] == 'Распределение эритроцитов по объему (RDW)'

# Задаем норму '10:18' в отфильтрованных строках в столбце 'Норма кол. показателя'
df.loc[rdw_mask, 'Норма кол. показателя'] = '10:18'

# Проверка изменений
print("Изменения в столбце 'Норма кол. показателя' для 'Распределение эритроцитов по объему (RDW)':")
print(df[rdw_mask][['Кол. лаб. показатель', 'Норма кол. показателя']])

Изменения в столбце 'Норма кол. показателя' для 'Распределение эритроцитов по объему (RDW)':
                            Кол. лаб. показатель Норма кол. показателя
23     Распределение эритроцитов по объему (RDW)                 10:18
53     Распределение эритроцитов по объему (RDW)                 10:18
83     Распределение эритроцитов по объему (RDW)                 10:18
111    Распределение эритроцитов по объему (RDW)                 10:18
194    Распределение эритроцитов по объему (RDW)                 10:18
...                                          ...                   ...
35969  Распределение эритроцитов по объему (RDW)                 10:18
36113  Распределение эритроцитов по объему (RDW)                 10:18
36175  Распределение эритроцитов по объему (RDW)                 10:18
36224  Распределение эритроцитов по объему (RDW)                 10:18
36284  Распределение эритроцитов по объему (RDW)                 10:18

[829 rows x 2 columns]


In [92]:
RDW_SD_df = df[df['Кол. лаб. показатель'] == 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)']
unique_RDW_SD = RDW_SD_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)':")
print(unique_RDW_SD)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)':
['37:54' '35.3:48.9' '35:48' '33.4:49.2' '33:49']


In [93]:
unique_ed_RDW_SD = RDW_SD_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Распределение эритроцитов по объему (RDW-SD)':")
print(unique_ed_RDW_SD)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Распределение эритроцитов по объему (RDW-SD)':
['фл']


In [94]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'RDW-SD'
rdw_sd_mask = df['Кол. лаб. показатель'] == 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)'

# Задаем норму '35:60 фл' в отфильтрованных строках в столбце 'Норма кол. показателя'
df.loc[rdw_sd_mask, 'Норма кол. показателя'] = '35:60'

# Проверка изменений
print("Изменения в столбце 'Норма кол. показателя' для 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)':")
print(df[rdw_sd_mask][['Кол. лаб. показатель', 'Норма кол. показателя']])

Изменения в столбце 'Норма кол. показателя' для 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)':
                                    Кол. лаб. показатель Норма кол. показателя
15     Распределение эритроцитов по объему, стандартн...                 35:60
142    Распределение эритроцитов по объему, стандартн...                 35:60
170    Распределение эритроцитов по объему, стандартн...                 35:60
297    Распределение эритроцитов по объему, стандартн...                 35:60
326    Распределение эритроцитов по объему, стандартн...                 35:60
...                                                  ...                   ...
36099  Распределение эритроцитов по объему, стандартн...                 35:60
36138  Распределение эритроцитов по объему, стандартн...                 35:60
36167  Распределение эритроцитов по объему, стандартн...                 35:60
36216  Распределение эритроцитов по объему, стандартн...                 35:60
36265  Расп

In [95]:
RDW_CV_df = df[df['Кол. лаб. показатель'] == 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)']
unique_RDW_CV = RDW_CV_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)':")
print(unique_RDW_CV)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)':
['10,5:18' '10:18']


In [96]:
unique_ed_RDW_CV = RDW_CV_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)':")
print(unique_ed_RDW_CV)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)':
['%']


In [97]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)'
rdw_cv_mask = df['Кол. лаб. показатель'] == 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)'

# Задаем норму '10:18' в отфильтрованных строках в столбце 'Норма кол. показателя'
df.loc[rdw_cv_mask, 'Норма кол. показателя'] = '10:18'

# Проверка изменений
print("Изменения в столбце 'Норма кол. показателя' для 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)':")
print(df[rdw_cv_mask][['Кол. лаб. показатель', 'Норма кол. показателя']])

Изменения в столбце 'Норма кол. показателя' для 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)':
                                    Кол. лаб. показатель Норма кол. показателя
14     Распределение эритроцитов по объему, коэффицие...                 10:18
141    Распределение эритроцитов по объему, коэффицие...                 10:18
169    Распределение эритроцитов по объему, коэффицие...                 10:18
296    Распределение эритроцитов по объему, коэффицие...                 10:18
327    Распределение эритроцитов по объему, коэффицие...                 10:18
...                                                  ...                   ...
36062  Распределение эритроцитов по объему, коэффицие...                 10:18
36074  Распределение эритроцитов по объему, коэффицие...                 10:18
36086  Распределение эритроцитов по объему, коэффицие...                 10:18
36098  Распределение эритроцитов по объему, коэффицие...                 10:18
36137  Распре

In [98]:
PDW_df = df[df['Кол. лаб. показатель'] == 'Распределение тромбоцитов по объему (PDW)']
unique_PDW = PDW_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Распределение тромбоцитов по объему (PDW)':")
print(unique_PDW)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Распределение тромбоцитов по объему (PDW)':
['9,3:16,7' '9:17' '15.0:17.0' '15:17' '_']


In [99]:
unique_ed_PDW = PDW_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Распределение тромбоцитов по объему (PDW)':")
print(unique_ed_PDW)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Распределение тромбоцитов по объему (PDW)':
['фл' '%' 'г/л']


In [100]:
PDW_ed_fl_df = df[(df['Кол. лаб. показатель'] == 'Распределение тромбоцитов по объему (PDW)') & (df['Ед. изм. кол. показателя'] == 'фл')]
PDW_ed_fl_df

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Возраст
27,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Распределение тромбоцитов по объему (PDW),13.5,фл,"9,3:16,7",Норм,NaN,NaN,NaN,0,62
57,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Распределение тромбоцитов по объему (PDW),12.9,фл,"9,3:16,7",Норм,NaN,NaN,NaN,0,62
146,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Распределение тромбоцитов по объему (PDW),17.3,фл,9:17,Повыш,NaN,NaN,NaN,0,62
171,24612d4d-c466-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Распределение тромбоцитов по объему (PDW),15.0,фл,9:17,Норм,NaN,NaN,NaN,0,85
182,24612d4d-c466-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Распределение тромбоцитов по объему (PDW),13.0,фл,"9,3:16,7",Норм,NaN,NaN,NaN,0,85
...,...,...,...,...,...,...,...,...,...,...,...,...
35729,69714728-b691-11ee-8606-005056880ecb,Другой уточненный лейкоз,Распределение тромбоцитов по объему (PDW),13.4,фл,"9,3:16,7",Норм,NaN,NaN,NaN,1,67
35748,69714728-b691-11ee-8606-005056880ecb,Другой уточненный лейкоз,Распределение тромбоцитов по объему (PDW),13.3,фл,"9,3:16,7",Норм,NaN,NaN,NaN,1,67
35777,69714728-b691-11ee-8606-005056880ecb,Другой уточненный лейкоз,Распределение тромбоцитов по объему (PDW),12.9,фл,"9,3:16,7",Норм,NaN,NaN,NaN,1,67
35796,69714728-b691-11ee-8606-005056880ecb,Другой уточненный лейкоз,Распределение тромбоцитов по объему (PDW),12.5,фл,"9,3:16,7",Норм,NaN,NaN,NaN,1,67


In [101]:
PDW_ed_gl_df = df[(df['Кол. лаб. показатель'] == 'Распределение тромбоцитов по объему (PDW)') & (df['Ед. изм. кол. показателя'] == 'г/л')]
PDW_ed_gl_df

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Возраст
14882,a6cc0de1-ac03-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Распределение тромбоцитов по объему (PDW),10.8,г/л,_,Норм,NaN,NaN,NaN,1,66
14931,a6cc0de1-ac03-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Распределение тромбоцитов по объему (PDW),9.8,г/л,_,Норм,NaN,NaN,NaN,1,66
16096,e150616f-b636-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Распределение тромбоцитов по объему (PDW),13.0,г/л,_,Норм,NaN,NaN,NaN,0,39
17738,7b9025cb-ae7c-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Распределение тромбоцитов по объему (PDW),12.0,г/л,_,Норм,NaN,NaN,NaN,0,72
17766,7b9025cb-ae7c-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Распределение тромбоцитов по объему (PDW),11.1,г/л,_,Норм,NaN,NaN,NaN,0,72


Судя по флагу -- это просто ошибка заполняющего и везде долждны быть проценты (норма в интернете указана в процентах)

In [102]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Распределение тромбоцитов по объему (PDW)'
pdw_mask = df['Кол. лаб. показатель'] == 'Распределение тромбоцитов по объему (PDW)'

# Заменяем единицы измерения на '%'
df.loc[pdw_mask, 'Ед. изм. кол. показателя'] = '%'

# Проверка изменений
print("Обновленные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Распределение тромбоцитов по объему (PDW)':")
print(df[pdw_mask][['Кол. лаб. показатель', 'Ед. изм. кол. показателя']].head())

Обновленные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Распределение тромбоцитов по объему (PDW)':
                          Кол. лаб. показатель Ед. изм. кол. показателя
27   Распределение тромбоцитов по объему (PDW)                        %
57   Распределение тромбоцитов по объему (PDW)                        %
146  Распределение тромбоцитов по объему (PDW)                        %
171  Распределение тромбоцитов по объему (PDW)                        %
182  Распределение тромбоцитов по объему (PDW)                        %


In [103]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)'
pdw_mask = df['Кол. лаб. показатель'] == 'Распределение тромбоцитов по объему (PDW)'

# Задаем норму '10:18' в отфильтрованных строках в столбце 'Норма кол. показателя'
df.loc[pdw_mask, 'Норма кол. показателя'] = '9,0:17,0'

# Проверка изменений
print("Изменения в столбце 'Норма кол. показателя' для 'Распределение тромбоцитов по объему (PDW)':")
print(df[pdw_mask][['Кол. лаб. показатель', 'Норма кол. показателя']])

Изменения в столбце 'Норма кол. показателя' для 'Распределение тромбоцитов по объему (PDW)':
                            Кол. лаб. показатель Норма кол. показателя
27     Распределение тромбоцитов по объему (PDW)              9,0:17,0
57     Распределение тромбоцитов по объему (PDW)              9,0:17,0
146    Распределение тромбоцитов по объему (PDW)              9,0:17,0
171    Распределение тромбоцитов по объему (PDW)              9,0:17,0
182    Распределение тромбоцитов по объему (PDW)              9,0:17,0
...                                          ...                   ...
35796  Распределение тромбоцитов по объему (PDW)              9,0:17,0
36166  Распределение тромбоцитов по объему (PDW)              9,0:17,0
36179  Распределение тромбоцитов по объему (PDW)              9,0:17,0
36215  Распределение тромбоцитов по объему (PDW)              9,0:17,0
36264  Распределение тромбоцитов по объему (PDW)              9,0:17,0

[657 rows x 2 columns]


In [104]:
LY_otn_df = df[df['Кол. лаб. показатель'] == 'Лимфоциты, относительное количество (LY%)']
unique_LY_otn = LY_otn_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Лимфоциты, относительное количество (LY%)':")
print(unique_LY_otn)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Лимфоциты, относительное количество (LY%)':
['18:44' '18,0:44,0 ' '18,0:44,0' '20:45']


In [105]:
unique_ed_LY_otn = LY_otn_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Лимфоциты, относительное количество (LY%)':")
print(unique_ed_LY_otn)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Лимфоциты, относительное количество (LY%)':
['%']


In [106]:
# Функция для определения нормы лимфоцитов на основе возраста и пола
def get_ly_otn_norm(age, gender):
    if age <= 1 / 12:  # до 1 месяца
        return '25:60'
    elif 1 / 12 < age <= 1:  # 2–12 месяцев
        return '40:70'
    elif 1 < age <= 5:  # 1–5 лет
        return '35:60'
    elif 5 < age <= 10:  # 5–10 лет
        return '30:50'
    elif 10 < age <= 14:  # 10–14 лет
        return '30:48'
    elif age > 14:  # > 14 лет
        return '22:45'
    else:
        return None  # В случае, если возраст не попадает ни в одну категорию

In [107]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Лимфоциты, относительное количество (LY%)'
ly_otn_mask = df['Кол. лаб. показатель'] == 'Лимфоциты, относительное количество (LY%)'

# Применяем функцию к нужным строкам и обновляем столбец 'Норма кол. показателя'
df.loc[ly_otn_mask, 'Норма кол. показателя'] = df[ly_otn_mask].apply(lambda row: get_ly_otn_norm(row['Возраст'], row['Пол']), axis=1)

# Проверка результатов
print("Обновленные значения 'Норма кол. показателя' для показателя 'Лимфоциты, относительное количество (LY%)':")
print(df[ly_otn_mask][['Возраст', 'Пол', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'Лимфоциты, относительное количество (LY%)':
     Возраст  Пол Норма кол. показателя
9         62    0                 22:45
29        62    0                 22:45
59        62    0                 22:45
93        62    0                 22:45
121       62    0                 22:45


In [109]:
LY_abs_df = df[df['Кол. лаб. показатель'] == 'Лимфоциты, абсолютное количество (LY#)']
unique_LY_abs = LY_abs_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Лимфоциты, абсолютное количество (LY#)':")
print(unique_LY_abs)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Лимфоциты, абсолютное количество (LY#)':
['1:3,7' '1,00:3,70' '1,00:3.70' '_' '1,0:3,7']


In [110]:
unique_ed_LY_abs = LY_abs_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Лимфоциты, абсолютное количество (LY#)':")
print(unique_ed_LY_abs)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Лимфоциты, абсолютное количество (LY#)':
['10*9/л']


In [111]:
# Функция для определения нормы абсолютного количества лимфоцитов на основе возраста и пола
def get_ly_abs_norm(age, gender):
    if age <= 1 / 12:  # до 1 месяца
        return '1,25:12,6'
    elif 1 / 12 < age <= 1:  # 2–12 месяцев
        return '2,2:12,3'
    elif 1 < age <= 5:  # 1–5 лет
        return '1,9:9,3'
    elif 5 < age <= 10:  # 5–10 лет
        return '1,3:7,3'
    elif 10 < age <= 14:  # 10–14 лет
        return '1,3:6,2'
    elif age > 14:  # > 14 лет (взрослые)
        return '1,0:4,0'
    else:
        return None  # В случае, если возраст не попадает ни в одну категорию

In [112]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Лимфоциты, абсолютное количество (LY#)'
ly_abs_mask = df['Кол. лаб. показатель'] == 'Лимфоциты, абсолютное количество (LY#)'

# Применяем функцию к нужным строкам и обновляем столбец 'Норма кол. показателя'
df.loc[ly_abs_mask, 'Норма кол. показателя'] = df[ly_abs_mask].apply(lambda row: get_ly_abs_norm(row['Возраст'], row['Пол']), axis=1)

# Проверка результатов
print("Обновленные значения 'Норма кол. показателя' для показателя 'Лимфоциты, абсолютное количество (LY#)':")
print(df[ly_abs_mask][['Возраст', 'Пол', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'Лимфоциты, абсолютное количество (LY#)':
     Возраст  Пол Норма кол. показателя
11        62    0               1,0:4,0
34        62    0               1,0:4,0
64        62    0               1,0:4,0
88        62    0               1,0:4,0
116       62    0               1,0:4,0


In [113]:
NE_otn_df = df[df['Кол. лаб. показатель'] == 'Нейтрофилы, относительное количество (NE%)']
unique_NE_otn = NE_otn_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Нейтрофилы, относительное количество (NE%)':")
print(unique_NE_otn)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Нейтрофилы, относительное количество (NE%)':
['45:72' '45,0:72,0' '41:75']


In [114]:
unique_ed_NE_otn = NE_otn_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Нейтрофилы, относительное количество (NE%)':")
print(unique_ed_NE_otn)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Нейтрофилы, относительное количество (NE%)':
['%']


In [115]:
# Функция для определения нормы NE на основе возраста
def get_ne_otn_norm(age, gender):
    if age <= 1 / 12:  # до 1 месяца
        return '31:55'
    elif 1 / 12 < age <= 1:  # 2–12 месяцев
        return '17:50'
    elif 1 < age <= 5:  # 1–5 лет
        return '30:60'
    elif 5 < age <= 10:  # 5–10 лет
        return '40:62'
    elif 10 < age <= 14:  # 10–14 лет
        return '44:65'
    elif age > 14:  # > 14 лет (взрослые)
        return '47:77'
    else:
        return None  # В случае, если возраст не попадает ни в одну категорию

In [116]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Нейтрофилы, относительное количество (NE%)'
ne_otn_mask = df['Кол. лаб. показатель'] == 'Нейтрофилы, относительное количество (NE%)'

# Применяем функцию к нужным строкам и обновляем столбец 'Норма кол. показателя'
df.loc[ne_otn_mask, 'Норма кол. показателя'] = df[ne_otn_mask].apply(lambda row: get_ne_otn_norm(row['Возраст'], row['Пол']), axis=1)

# Проверка результатов
print("Обновленные значения 'Норма кол. показателя' для показателя 'Нейтрофилы, относительное количество (NE%)':")
print(df[ne_otn_mask][['Возраст', 'Пол', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'Нейтрофилы, относительное количество (NE%)':
     Возраст  Пол Норма кол. показателя
8         62    0                 47:77
28        62    0                 47:77
58        62    0                 47:77
92        62    0                 47:77
120       62    0                 47:77


In [117]:
ne_abs_df = df[df['Кол. лаб. показатель'] == 'Нейтрофилы, абсолютное количество (NE#)']
unique_ne_abs = ne_abs_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Нейтрофилы, абсолютное количество (NE#)':")
print(unique_ne_abs)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Нейтрофилы, абсолютное количество (NE#)':
['1,5:7' '1,50:7,00' '_' '1,5:7,0']


In [118]:
unique_ed_ne_abs = ne_abs_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Нейтрофилы, абсолютное количество (NE#)':")
print(unique_ed_ne_abs)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Нейтрофилы, абсолютное количество (NE#)':
['10*9/л']


In [119]:
# Функция для определения нормы NE# на основе возраста
def get_ne_abs_norm(age, gender):
    if age <= 1 / 12:  # до 1 месяца
        return '1,5:11,5'
    elif 1 / 12 < age <= 1:  # 2–12 месяцев
        return '0,9:8,8'
    elif 1 < age <= 5:  # 1–5 лет
        return '1,5:9,3'
    elif 5 < age <= 10:  # 5–10 лет
        return '1,8:8,1'
    elif 10 < age <= 14:  # 10–14 лет
        return '1,9:7,5'
    elif age > 14:  # > 14 лет (взрослые)
        return '1,8:7,2'
    else:
        return None  # В случае, если возраст не попадает ни в одну категорию

In [120]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Нейтрофилы, абсолютное количество (NE#)'
ne_abs_mask = df['Кол. лаб. показатель'] == 'Нейтрофилы, абсолютное количество (NE#)'

# Применяем функцию к нужным строкам и обновляем столбец 'Норма кол. показателя'
df.loc[ne_abs_mask, 'Норма кол. показателя'] = df[ne_abs_mask].apply(lambda row: get_ne_abs_norm(row['Возраст'], row['Пол']), axis=1)

# Проверка результатов
print("Обновленные значения 'Норма кол. показателя' для показателя 'Нейтрофилы, абсолютное количество (NE#)':")
print(df[ne_abs_mask][['Возраст', 'Пол', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'Нейтрофилы, абсолютное количество (NE#)':
     Возраст  Пол Норма кол. показателя
10        62    0               1,8:7,2
33        62    0               1,8:7,2
63        62    0               1,8:7,2
87        62    0               1,8:7,2
115       62    0               1,8:7,2


In [121]:
eo_otn_df = df[df['Кол. лаб. показатель'] == 'Эозинофилы, относительное количество (EO%)']
unique_eo_otn = eo_otn_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Эозинофилы, относительное количество (EO%)':")
print(unique_eo_otn)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Эозинофилы, относительное количество (EO%)':
['0:5' '0,0:5,0']


In [122]:
unique_ed_eo_otn = eo_otn_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Эозинофилы, относительное количество (EO%)':")
print(unique_ed_eo_otn)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Эозинофилы, относительное количество (EO%)':
['%']


In [123]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Эозинофилы, относительное количество (EO%)'
eo_otn_mask = df['Кол. лаб. показатель'] == 'Эозинофилы, относительное количество (EO%)'

# Задаем норму '0,8:6,2' для всех выбранных строк
df.loc[eo_otn_mask, 'Норма кол. показателя'] = '0,0:5,0'

# Проверка результата
print("Обновленные значения 'Норма кол. показателя' для показателя 'Эозинофилы, относительное количество (EO%)':")
print(df[eo_otn_mask][['Кол. лаб. показатель', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'Эозинофилы, относительное количество (EO%)':
                           Кол. лаб. показатель Норма кол. показателя
31   Эозинофилы, относительное количество (EO%)               0,0:5,0
61   Эозинофилы, относительное количество (EO%)               0,0:5,0
95   Эозинофилы, относительное количество (EO%)               0,0:5,0
123  Эозинофилы, относительное количество (EO%)               0,0:5,0
186  Эозинофилы, относительное количество (EO%)               0,0:5,0


In [124]:
eo_abs_df = df[df['Кол. лаб. показатель'] == 'Эозинофилы, абсолютное количество (EO#)']
unique_eo_abs = eo_abs_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Эозинофилы, абсолютное количество (EO#)':")
print(unique_eo_abs)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Эозинофилы, абсолютное количество (EO#)':
['0:0,4' '0,00:0,40']


In [125]:
unique_ed_eo_abs = eo_abs_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Эозинофилы, абсолютное количество (EO#))':")
print(unique_ed_eo_abs)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Эозинофилы, абсолютное количество (EO#))':
['10*9/л']


In [126]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Эозинофилы, абсолютное количество (EO#)   '
eo_abs_mask = df['Кол. лаб. показатель'] == 'Эозинофилы, абсолютное количество (EO#)'

# Задаем норму '0,8:6,2' для всех выбранных строк
df.loc[eo_abs_mask, 'Норма кол. показателя'] = '0,00:0,40'

# Проверка результата
print("Обновленные значения 'Норма кол. показателя' для показателя 'Эозинофилы, абсолютное количество (EO#)':")
print(df[eo_abs_mask][['Кол. лаб. показатель', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'Эозинофилы, абсолютное количество (EO#)':
                        Кол. лаб. показатель Норма кол. показателя
36   Эозинофилы, абсолютное количество (EO#)             0,00:0,40
66   Эозинофилы, абсолютное количество (EO#)             0,00:0,40
90   Эозинофилы, абсолютное количество (EO#)             0,00:0,40
118  Эозинофилы, абсолютное количество (EO#)             0,00:0,40
192  Эозинофилы, абсолютное количество (EO#)             0,00:0,40


In [127]:
baz_otn_df = df[df['Кол. лаб. показатель'] == 'Базофилы, относительное количество (BA%)']
unique_baz_otn = baz_otn_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Базофилы, относительное количество (BA%)':")
print(unique_baz_otn)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Базофилы, относительное количество (BA%)':
['0:2' '0,0:2,0']


In [128]:
unique_ed_baz_otn = baz_otn_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Базофилы, относительное количество (BA%)':")
print(unique_ed_baz_otn)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Базофилы, относительное количество (BA%)':
['%']


In [129]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Эозинофилы, относительное количество (EO%)'
baz_otn_mask = df['Кол. лаб. показатель'] == 'Базофилы, относительное количество (BA%)'

# Задаем норму '0,8:6,2' для всех выбранных строк
df.loc[baz_otn_mask, 'Норма кол. показателя'] = '0,0:2,0'

# Проверка результата
print("Обновленные значения 'Норма кол. показателя' для показателя 'Базофилы, относительное количество (BA%)':")
print(df[baz_otn_mask][['Кол. лаб. показатель', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'Базофилы, относительное количество (BA%)':
                         Кол. лаб. показатель Норма кол. показателя
32   Базофилы, относительное количество (BA%)               0,0:2,0
62   Базофилы, относительное количество (BA%)               0,0:2,0
96   Базофилы, относительное количество (BA%)               0,0:2,0
124  Базофилы, относительное количество (BA%)               0,0:2,0
187  Базофилы, относительное количество (BA%)               0,0:2,0


In [130]:
baz_abs_df = df[df['Кол. лаб. показатель'] == 'Базофилы, абсолютное количество (BA#)']
unique_baz_abs = baz_abs_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Базофилы, абсолютное количество (BA#)':")
print(unique_baz_abs)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Базофилы, абсолютное количество (BA#)':
['0:0,1' '0,00:0,10' '0:0.1']


In [131]:
unique_ed_baz_abs = baz_abs_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Базофилы, абсолютное количество (BA#)':")
print(unique_ed_baz_abs)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Базофилы, абсолютное количество (BA#)':
['10*9/л']


In [132]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Базофилы, абсолютное количество (BA#)  '
baz_abs_mask = df['Кол. лаб. показатель'] == 'Базофилы, абсолютное количество (BA#)'

# Задаем норму '0,8:6,2' для всех выбранных строк
df.loc[baz_abs_mask, 'Норма кол. показателя'] = '0,00:0,10'

# Проверка результата
print("Обновленные значения 'Норма кол. показателя' для показателя 'Базофилы, абсолютное количество (BA#)':")
print(df[baz_abs_mask][['Кол. лаб. показатель', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'Базофилы, абсолютное количество (BA#)':
                      Кол. лаб. показатель Норма кол. показателя
37   Базофилы, абсолютное количество (BA#)             0,00:0,10
67   Базофилы, абсолютное количество (BA#)             0,00:0,10
91   Базофилы, абсолютное количество (BA#)             0,00:0,10
119  Базофилы, абсолютное количество (BA#)             0,00:0,10
193  Базофилы, абсолютное количество (BA#)             0,00:0,10


In [133]:
mono_otn_df = df[df['Кол. лаб. показатель'] == 'Моноциты, относительное количество (MO%)']
unique_mono_otn = mono_otn_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Моноциты, относительное количество (MO%)':")
print(unique_mono_otn)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Моноциты, относительное количество (MO%)':
['2:12' '2,0:12,0' '2,0:12,0 ']


In [134]:
unique_ed_mono_otn = mono_otn_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Моноциты, относительное количество (MO%)':")
print(unique_ed_mono_otn)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Моноциты, относительное количество (MO%)':
['%']


In [135]:
# Функция для определения нормы относительного количества моноцитов на основе возраста
def get_mono_otn_norm(age):
    if age <= 1 / 12:  # до 1 месяца
        return '5:15'
    elif 1 / 12 < age <= 1:  # 2–12 месяцев
        return '4:10'
    elif 1 < age <= 14:  # 1–14 лет
        return '3:11'
    elif age > 14:  # > 14 лет (взрослые)
        return '2:12'
    else:
        return None  # В случае, если возраст не попадает ни в одну категорию


In [136]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Моноциты, относительное количество (MO%))'
mono_otn_mask = df['Кол. лаб. показатель'] == 'Моноциты, относительное количество (MO%)'

# Применяем функцию к нужным строкам и обновляем столбец 'Норма кол. показателя'
df.loc[mono_otn_mask, 'Норма кол. показателя'] = df[mono_otn_mask].apply(lambda row: get_mono_otn_norm(row['Возраст']), axis=1)

# Проверка результатов
print("Обновленные значения 'Норма кол. показателя' для показателя 'Моноциты, относительное количество (MO%)':")
print(df[mono_otn_mask][['Возраст', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'Моноциты, относительное количество (MO%)':
     Возраст Норма кол. показателя
30        62                  2:12
60        62                  2:12
94        62                  2:12
122       62                  2:12
185       85                  2:12


In [137]:
mono_abs_df = df[df['Кол. лаб. показатель'] == 'Моноциты, абсолютное количество (MO#)']
unique_mono_abs = mono_abs_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Моноциты, абсолютное количество (MO#)':")
print(unique_mono_abs)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Моноциты, абсолютное количество (MO#)':
['0,1:0,7' '0,10:0,70' '0:0,7']


In [138]:
unique_ed_mono_abs = mono_abs_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Моноциты, абсолютное количество (MO#)':")
print(unique_ed_mono_abs)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Моноциты, абсолютное количество (MO#)':
['10*9/л']


In [139]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Моноциты, абсолютное количество (MO#)'
mono_abs_mask = df['Кол. лаб. показатель'] == 'Моноциты, абсолютное количество (MO#)'

# Задаем норму '0,8:6,2' для всех выбранных строк
df.loc[mono_abs_mask, 'Норма кол. показателя'] = '0,10:0,70'

# Проверка результата
print("Обновленные значения 'Норма кол. показателя' для показателя 'Моноциты, абсолютное количество (MO#)':")
print(df[mono_abs_mask][['Кол. лаб. показатель', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'Моноциты, абсолютное количество (MO#)':
                      Кол. лаб. показатель Норма кол. показателя
35   Моноциты, абсолютное количество (MO#)             0,10:0,70
65   Моноциты, абсолютное количество (MO#)             0,10:0,70
89   Моноциты, абсолютное количество (MO#)             0,10:0,70
117  Моноциты, абсолютное количество (MO#)             0,10:0,70
191  Моноциты, абсолютное количество (MO#)             0,10:0,70


In [140]:
mxd_otn_df = df[df['Кол. лаб. показатель'] == 'Смешанная фракция, относительное количество (MXD%)']
unique_mxd_otn = mxd_otn_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Смешанная фракция, относительное количество (MXD%)':")
print(unique_mxd_otn)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Смешанная фракция, относительное количество (MXD%)':
['5:10']


In [141]:
unique_ed_mxd_otn = mxd_otn_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Смешанная фракция, относительное количество (MXD%)':")
print(unique_ed_mxd_otn)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Смешанная фракция, относительное количество (MXD%)':
['%']


In [142]:
soe_df = df[df['Кол. лаб. показатель'] == 'СОЭ по Вестергрену']
unique_soe = soe_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'СОЭ по Вестергрену':")
print(unique_soe)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'СОЭ по Вестергрену':
['2:20' 'Общая норма']


In [143]:
unique_ed_soe = soe_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'СОЭ по Вестергрену':")
print(unique_ed_soe)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'СОЭ по Вестергрену':
['мм/час']


In [144]:
# Функция для определения нормы СОЭ по Вестергрену на основе возраста и пола
def get_soe_norm(age, gender):
    if age < 10:  # Дети до 10 лет
        return '2:10'
    elif gender == 1:  # Мужчины
        if age < 50:
            return '1:15'
        else:
            return '2:20'
    elif gender == 0:  # Женщины
        if age < 50:
            return '2:20'
        else:
            return '2:30'
    else:
        return None  # В случае, если возраст или пол не указаны

In [145]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'СОЭ по Вестергрену'
soe_mask = df['Кол. лаб. показатель'] == 'СОЭ по Вестергрену'

# Применяем функцию к нужным строкам и обновляем столбец 'Норма кол. показателя'
df.loc[soe_mask, 'Норма кол. показателя'] = df[soe_mask].apply(lambda row: get_soe_norm(row['Возраст'], row['Пол']), axis=1)

# Проверка результатов
print("Обновленные значения 'Норма кол. показателя' для показателя 'СОЭ по Вестергрену':")
print(df[soe_mask][['Возраст', 'Пол', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'СОЭ по Вестергрену':
     Возраст  Пол Норма кол. показателя
39        62    0                  2:30
69        62    0                  2:30
98        62    0                  2:30
126       62    0                  2:30
196       85    0                  2:30


In [146]:
seg_neu_df = df[df['Кол. лаб. показатель'] == 'Сегментоядерные нейтрофилы (%)']
unique_seg_neu = seg_neu_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Сегментоядерные нейтрофилы (%)':")
print(unique_seg_neu)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Сегментоядерные нейтрофилы (%)':
['45:72']


In [147]:
unique_ed_seg_neu = seg_neu_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Сегментоядерные нейтрофилы (%)':")
print(unique_ed_seg_neu)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Сегментоядерные нейтрофилы (%)':
['%']


In [148]:
unique_znach_seg_neu = seg_neu_df['Значение кол. показателя'].unique()
print("Уникальные значения в столбце 'Значение кол. показателя' для показателя 'Сегментоядерные нейтрофилы (%)':")
print(unique_znach_seg_neu)

Уникальные значения в столбце 'Значение кол. показателя' для показателя 'Сегментоядерные нейтрофилы (%)':
[20.  50.  90.  18.  59.  54.  22.  76.   6.  24.  39.   3.  17.  57.
 67.  62.  25.  32.  13.  28.  55.  56.  15.  64.  14.  52.  61.  60.
 49.  27.  11.  65.  51.  72.   5.  26.  45.  10.  41.  43.  36.  81.
 48.  30.  79.  73.  35.  38.  68.  77.   7.   4.  12.  58.  52.5 37.
 71.  46.   8.  84.  23.   9.   1.   0.  19.  47.  75.  21.  40.  16.
 80.   4.5 29.  63.  86.  70.  69.  34.  42.  44.  31.  74.  33.  78.
 53.  66.  33.5 36.5 85.  88.  87.  82.  58.5 65.5 71.5 39.5  1.5 83.
  2.  89. ]


## АТЕНШН 
Сегментоядерные нейтрофилы надо переделать как то по умному, сейчас пока не соображу... 
смысл в том что может быть я на каком то этапе или составители датасета запихнули сюда как процентное содержание, так и абсолютное содержание. 

Согласно данным на сайте: https://gemotest.ru/info/spravochnik/analizy/neytrofily/

все, что ниже 17 (а лучше начиная с 16) -- это абсолютное содержание и нужно такие значения переназвать на `'Сегментоядерные нейтрофилы (#)'`, для них будет другая норма. Норму тоже нужно взять с этого сайта, который выше. 


!!!! с другой стороны это них не так

In [149]:
# Фильтруем строки, где 'Кол. лаб. показатель' равен 'Сегментоядерные нейтрофилы (%)' и 'Значение кол. показателя' меньше 17
filtered_df = df[(df['Кол. лаб. показатель'] == 'Сегментоядерные нейтрофилы (%)') & (df['Значение кол. показателя'] < 17)]
filtered_df

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Возраст
515,b0a85bb2-3404-11ed-ab56-0050568844e6,Плазмоклеточный лейкоз,Сегментоядерные нейтрофилы (%),6.0,%,45:72,Пониж,Комментарий,плазматические клетки с омоложением ядра,_,0,62
634,b0a85bb2-3404-11ed-ab56-0050568844e6,Плазмоклеточный лейкоз,Сегментоядерные нейтрофилы (%),3.0,%,45:72,Пониж,NaN,NaN,NaN,0,62
766,b0a85bb2-3404-11ed-ab56-0050568844e6,Плазмоклеточный лейкоз,Сегментоядерные нейтрофилы (%),6.0,%,45:72,Пониж,NaN,NaN,NaN,0,62
1341,16e21cb0-ea56-11ed-ab67-0050568844e6,Другой уточненный лейкоз,Сегментоядерные нейтрофилы (%),13.0,%,45:72,Пониж,Комментарий,из 24 моноцитоидных элементов 16 промоноцитов,_,1,66
1703,eb8b7ced-2860-11ed-ab56-0050568844e6,Острый лимфобластный лейкоз,Сегментоядерные нейтрофилы (%),15.0,%,45:72,Пониж,NaN,NaN,NaN,1,75
...,...,...,...,...,...,...,...,...,...,...,...,...
28395,965719ab-f0fd-11e9-80bd-901b0e633689,Острый миелоидный лейкоз,Сегментоядерные нейтрофилы (%),2.0,%,45:72,Пониж,Комментарий,мононуклеоидные элементы с более грой структур...,_,1,90
28424,965719ab-f0fd-11e9-80bd-901b0e633689,Острый миелоидный лейкоз,Сегментоядерные нейтрофилы (%),2.0,%,45:72,Пониж,NaN,NaN,NaN,1,90
28450,965719ab-f0fd-11e9-80bd-901b0e633689,Острый миелоидный лейкоз,Сегментоядерные нейтрофилы (%),3.0,%,45:72,Пониж,NaN,NaN,NaN,1,90
36192,6166e156-c1af-11ed-8602-005056880ecb,Острый лейкоз неуточненного клеточного типа,Сегментоядерные нейтрофилы (%),2.0,%,45:72,Пониж,Комментарий,"лимфоидные элементы средних размеров,с ядрами ...",_,1,42


In [150]:
# Получаем уникальные значения в столбце 'Флаг нормы кол. показателя' для отфильтрованных строк
unique_flags = filtered_df['Флаг нормы кол. показателя'].unique()
unique_flags

array(['Пониж'], dtype=object)

In [152]:
pal_neu_df = df[df['Кол. лаб. показатель'] == 'Палочкоядерные нейтрофилы (%)']
unique_pal_neu = pal_neu_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Палочкоядерные нейтрофилы (%)':")
print(unique_pal_neu)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Палочкоядерные нейтрофилы (%)':
['1:6']


In [153]:
unique_ed_pal_neu = pal_neu_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Палочкоядерные нейтрофилы (%)':")
print(unique_ed_pal_neu)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Палочкоядерные нейтрофилы (%)':
['%']


In [154]:
leico_ly_df = df[df['Кол. лаб. показатель'] == 'Лимфоциты (лейкоцитарная формула, %)']
unique_leico_ly = leico_ly_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Лимфоциты (лейкоцитарная формула, %)':")
print(unique_leico_ly)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Лимфоциты (лейкоцитарная формула, %)':
['18:44']


In [155]:
unique_ed_leico_ly= leico_ly_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Лимфоциты (лейкоцитарная формула, %)':")
print(unique_ed_leico_ly)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Лимфоциты (лейкоцитарная формула, %)':
['%']


In [156]:
leico_proly_df = df[df['Кол. лаб. показатель'] == 'Пролимфоциты (лейкоцитарная формула, %)']
unique_leico_proly = leico_proly_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Пролимфоциты (лейкоцитарная формула, %)':")
print(unique_leico_proly)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Пролимфоциты (лейкоцитарная формула, %)':
['0:0']


In [157]:
unique_ed_leico_proly = leico_proly_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Пролимфоциты (лейкоцитарная формула, %)':")
print(unique_ed_leico_proly)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Пролимфоциты (лейкоцитарная формула, %)':
['%']


In [158]:
unique_znach_leico_proly = leico_proly_df['Значение кол. показателя'].unique()
print("Уникальные значения в столбце 'Значение кол. показателя' для показателя 'Пролимфоциты (лейкоцитарная формула, %)':")
print(unique_znach_leico_proly)

Уникальные значения в столбце 'Значение кол. показателя' для показателя 'Пролимфоциты (лейкоцитарная формула, %)':
[ 2.  1.  0. 12.]


In [159]:
# Подсчет количества записей с 'Кол. лаб. показатель' равным 'Пролимфоциты (лейкоцитарная формула, %)'
prolymphocyte_count = df[df['Кол. лаб. показатель'] == 'Пролимфоциты (лейкоцитарная формула, %)'].shape[0]
prolymphocyte_count

16

странная хрень. 

в интернете быстрый поиск не дал результатов что это за зверь, да и показателей таких в датасете не много --- Алина приняла решение их удалить

In [160]:
# Удаляем строки, где 'Кол. лаб. показатель' равен 'Пролимфоциты (лейкоцитарная формула, %)'
df = df[df['Кол. лаб. показатель'] != 'Пролимфоциты (лейкоцитарная формула, %)']

# Проверяем, что строки успешно удалены
print("Строки с показателем 'Пролимфоциты (лейкоцитарная формула, %)' удалены.")

Строки с показателем 'Пролимфоциты (лейкоцитарная формула, %)' удалены.


In [161]:
leico_mielo_df = df[df['Кол. лаб. показатель'] == 'Миелоциты (лейкоцитарная формула, %)']
unique_leico_mielo = leico_mielo_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Миелоциты (лейкоцитарная формула, %)':")
print(unique_leico_mielo)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Миелоциты (лейкоцитарная формула, %)':
['0:0']


In [162]:
unique_ed_leico_mielo = leico_mielo_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Миелоциты (лейкоцитарная формула, %)':")
print(unique_ed_leico_mielo)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Миелоциты (лейкоцитарная формула, %)':
['%']


In [163]:
unique_znach_leico_mielo = leico_mielo_df['Значение кол. показателя'].unique()
print("Уникальные значения в столбце 'Значение кол. показателя' для показателя 'Миелоциты (лейкоцитарная формула, %)':")
print(unique_znach_leico_mielo)

Уникальные значения в столбце 'Значение кол. показателя' для показателя 'Миелоциты (лейкоцитарная формула, %)':
[ 4.   5.   2.   1.   8.   6.  12.   3.   4.5  9.   7.   3.5 10.  13.
 10.5 14.   0.   7.5  5.5 11.   1.5  0.5 16.  20.  18.  35.   6.5 14.5
  0.3 15.  19. ]


In [165]:
# Подсчет количества записей с 'Кол. лаб. показатель' равным 'Миелоциты (лейкоцитарная формула, %)'
mielocyte_count = df[df['Кол. лаб. показатель'] == 'Миелоциты (лейкоцитарная формула, %)'].shape[0]
mielocyte_count

316

В норме в крови не обнаруживаются миелоциты и метамиелоциты, а доля палочкоядерных нейтрофилов не превышает 5%. Но если в организме развивается инфекция, то иммунная система бросает в бой все свои силы, в том числе и несозревшие нейтрофилы. 

предлагаю убрать миелоциты вообще, просто не будем охватывать эту сферу болезней так глубоко. 

In [166]:
# Удаляем строки, где 'Кол. лаб. показатель' равен 'Миелоциты (лейкоцитарная формула, %)'
df = df[df['Кол. лаб. показатель'] != 'Миелоциты (лейкоцитарная формула, %)']

# Проверяем, что строки успешно удалены
print("Строки с показателем 'Миелоциты (лейкоцитарная формула, %)' удалены.")

Строки с показателем 'Миелоциты (лейкоцитарная формула, %)' удалены.


In [167]:
leico_promielo_df = df[df['Кол. лаб. показатель'] == 'Промиелоциты (лейкоцитарная формула, %)']
unique_leico_promielo = leico_promielo_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Промиелоциты (лейкоцитарная формула, %)':")
print(unique_leico_promielo)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Промиелоциты (лейкоцитарная формула, %)':
['0:0']


In [170]:
unique_ed_leico_promielo = leico_promielo_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Промиелоциты (лейкоцитарная формула, %)':")
print(unique_ed_leico_promielo)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Промиелоциты (лейкоцитарная формула, %)':
['%']


In [171]:
unique_znach_leico_promielo= leico_promielo_df['Значение кол. показателя'].unique()
print("Уникальные значения в столбце 'Значение кол. показателя' для показателя 'Промиелоциты (лейкоцитарная формула, %)':")
print(unique_znach_leico_promielo)

Уникальные значения в столбце 'Значение кол. показателя' для показателя 'Промиелоциты (лейкоцитарная формула, %)':
[0. 1.]


In [172]:
# Подсчет количества записей с 'Кол. лаб. показатель' равным 'Миелоциты (лейкоцитарная формула, %)'
promielocyte_count = df[df['Кол. лаб. показатель'] == 'Промиелоциты (лейкоцитарная формула, %)'].shape[0]
promielocyte_count

21

In [173]:
# Удаляем строки, где 'Кол. лаб. показатель' равен 'Миелоциты (лейкоцитарная формула, %)'
df = df[df['Кол. лаб. показатель'] != 'Промиелоциты (лейкоцитарная формула, %)']

# Проверяем, что строки успешно удалены
print("Строки с показателем 'Промиелоциты (лейкоцитарная формула, %)' удалены.")

Строки с показателем 'Промиелоциты (лейкоцитарная формула, %)' удалены.


In [174]:
leico_mono_df = df[df['Кол. лаб. показатель'] == 'Моноциты (лейкоцитарная формула, %)']
unique_leico_mono = leico_mono_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Моноциты (лейкоцитарная формула, %)':")
print(unique_leico_mono)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Моноциты (лейкоцитарная формула, %)':
['2:12' '2:10']


In [175]:
unique_ed_leico_mono = leico_mono_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Моноциты (лейкоцитарная формула, %)':")
print(unique_ed_leico_mono)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Моноциты (лейкоцитарная формула, %)':
['%']


In [176]:
# Создаем маску для строк, где 'Кол. лаб. показатель' равен 'Моноциты (лейкоцитарная формула, %)'
leico_mono_mask = df['Кол. лаб. показатель'] == 'Моноциты (лейкоцитарная формула, %)'

# Задаем норму '0,8:6,2' для всех выбранных строк
df.loc[leico_mono_mask, 'Норма кол. показателя'] = '2:12'

# Проверка результата
print("Обновленные значения 'Норма кол. показателя' для показателя 'Моноциты (лейкоцитарная формула, %)':")
print(df[leico_mono_mask][['Кол. лаб. показатель', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'Моноциты (лейкоцитарная формула, %)':
                    Кол. лаб. показатель Норма кол. показателя
44   Моноциты (лейкоцитарная формула, %)                  2:12
74   Моноциты (лейкоцитарная формула, %)                  2:12
103  Моноциты (лейкоцитарная формула, %)                  2:12
132  Моноциты (лейкоцитарная формула, %)                  2:12
204  Моноциты (лейкоцитарная формула, %)                  2:12


In [177]:
leico_eo_df = df[df['Кол. лаб. показатель'] == 'Эозинофилы (лейкоцитарная формула, %)']
unique_leico_eo = leico_eo_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Эозинофилы (лейкоцитарная формула, %)':")
print(unique_leico_eo)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Эозинофилы (лейкоцитарная формула, %)':
['0:5']


In [178]:
unique_ed_leico_eo = leico_eo_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Эозинофилы (лейкоцитарная формула, %)':")
print(unique_ed_leico_eo)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Эозинофилы (лейкоцитарная формула, %)':
['%']


In [179]:
leico_baz_df = df[df['Кол. лаб. показатель'] == 'Базофилы (лейкоцитарная формула, %)']
unique_leico_baz = leico_baz_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Базофилы (лейкоцитарная формула, %)':")
print(unique_leico_baz)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Базофилы (лейкоцитарная формула, %)':
['0:2']


In [180]:
unique_ed_leico_baz = leico_baz_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Базофилы (лейкоцитарная формула, %)':")
print(unique_ed_leico_baz)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Базофилы (лейкоцитарная формула, %)':
['%']


In [181]:
leico_retic_df = df[df['Кол. лаб. показатель'] == 'Ретикулоциты, относительное количество (%)']
unique_leico_retic = leico_retic_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Ретикулоциты, относительное количество (%)':")
print(unique_leico_retic)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Ретикулоциты, относительное количество (%)':
['0,2:2' '0,59:2,07' '0,2:1,2']


In [182]:
unique_ed_leico_retic = leico_retic_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Ретикулоциты, относительное количество (%)':")
print(unique_ed_leico_retic)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Ретикулоциты, относительное количество (%)':
['%']


In [183]:
# Функция для определения нормы "Ретикулоциты, относительное количество (%)" на основе возраста и пола
def get_retic_otn_norm(age, gender):
    if gender == 0:  # Женский
        if age < 2 / 52:  # Меньше 2 недель
            return '0,15:1,5'
        elif 2 / 52 <= age < 1 / 12:  # 2 недели – 1 месяц
            return '0,45:1,4'
        elif 1 / 12 <= age < 2 / 12:  # 1–2 месяца
            return '0,45:2,1'
        elif 2 / 12 <= age < 6 / 12:  # 2–6 месяцев
            return '0,25:0,9'
        elif 6 / 12 <= age < 2:  # 6 месяцев – 2 года
            return '0,2:1'
        elif 2 <= age < 6:  # 2–6 лет
            return '0,2:0,7'
        elif 6 <= age < 12:  # 6–12 лет
            return '0,2:1,3'
        elif 12 <= age < 18:  # 12–18 лет
            return '0,12:2,05'
        elif age >= 18:  # Больше 18 лет
            return '0,59:2,07'
    elif gender == 1:  # Мужской
        if age < 2 / 52:  # Меньше 2 недель
            return '0,15:1,5'
        elif 2 / 52 <= age < 1 / 12:  # 2 недели – 1 месяц
            return '0,45:1,4'
        elif 1 / 12 <= age < 2 / 12:  # 1–2 месяца
            return '0,45:2,1'
        elif 2 / 12 <= age < 6 / 12:  # 2–6 месяцев
            return '0,25:0,9'
        elif 6 / 12 <= age < 2:  # 6 месяцев – 2 года
            return '0,2:1'
        elif 2 <= age < 6:  # 2–6 лет
            return '0,2:0,7'
        elif 6 <= age < 12:  # 6–12 лет
            return '0,2:1,3'
        elif 12 <= age < 18:  # 12–18 лет
            return '0,24:1,7'
        elif age >= 18:  # Больше 18 лет
            return '0,67:1,92'
    return None  # Если возраст или пол не попадают ни в одну категорию


In [184]:
# Маска для строк с показателем "Ретикулоциты, относительное количество (%)"
retic_otn_mask = df['Кол. лаб. показатель'] == 'Ретикулоциты, относительное количество (%)'

# Применяем функцию к нужным строкам и обновляем столбец 'Норма кол. показателя'
df.loc[retic_otn_mask, 'Норма кол. показателя'] = df[retic_otn_mask].apply(
    lambda row: get_retic_otn_norm(row['Возраст'], row['Пол']), axis=1
)

# Проверка результатов
print("Обновленные значения 'Норма кол. показателя' для показателя 'Ретикулоциты, относительное количество (%)':")
print(df[retic_otn_mask][['Возраст', 'Пол', 'Норма кол. показателя']].head())

Обновленные значения 'Норма кол. показателя' для показателя 'Ретикулоциты, относительное количество (%)':
      Возраст  Пол Норма кол. показателя
631        62    0             0,59:2,07
1755       62    0             0,59:2,07
3712       85    0             0,59:2,07
5417       85    0             0,59:2,07
7971       72    0             0,59:2,07


In [ ]:
Ретикулоциты, абсолютное количество (#) 

In [185]:
retic_abs_df = df[df['Кол. лаб. показатель'] == 'Ретикулоциты, абсолютное количество (#)']
unique_retic_abs = retic_abs_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Ретикулоциты, абсолютное количество (#)':")
print(unique_retic_abs)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Ретикулоциты, абсолютное количество (#)':
['17:70' '0,2:2,0' '0,2:2' '17:63,8']


## АТЕНШЕН 

сюда явно попали показатели, которые должны были быть в `'Ретикулоциты, относительное количество (%)'`

у меня сейяас нет сил думать как их туда переправить, так что до следующего

In [ ]:
Нормобласты, относительное количество (%) 

In [186]:
norm_otn_df = df[df['Кол. лаб. показатель'] == 'Нормобласты, относительное количество (%)']
unique_norm_otn = norm_otn_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Нормобласты, относительное количество (%)':")
print(unique_norm_otn)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Нормобласты, относительное количество (%)':
['_']


In [187]:
unique_ed_norm_otn = norm_otn_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Нормобласты, относительное количество (%)':")
print(unique_ed_norm_otn)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Нормобласты, относительное количество (%)':
['%']


In [188]:
unique_znach_norm_otn= norm_otn_df['Значение кол. показателя'].unique()
print("Уникальные значения в столбце 'Значение кол. показателя' для показателя 'Нормобласты, относительное количество (%)':")
print(unique_znach_norm_otn)

Уникальные значения в столбце 'Значение кол. показателя' для показателя 'Нормобласты, относительное количество (%)':
[0.]


In [189]:
# Подсчет количества записей с 'Кол. лаб. показатель' равным 'Нормобласты, относительное количество (%)'
norm_otn_count = df[df['Кол. лаб. показатель'] == 'Нормобласты, относительное количество (%)'].shape[0]
norm_otn_count

18

In [190]:
# Удаляем строки, где 'Кол. лаб. показатель' равен 'Миелоциты (лейкоцитарная формула, %)'
df = df[df['Кол. лаб. показатель'] != 'Нормобласты, относительное количество (%)']

# Проверяем, что строки успешно удалены
print("Строки с показателем 'Нормобласты, относительное количество (%)' удалены.")

Строки с показателем 'Нормобласты, относительное количество (%)' удалены.


In [ ]:
Нормобласты, абсолютное количество (#)  

In [191]:
norm_abs_df = df[df['Кол. лаб. показатель'] == 'Нормобласты, абсолютное количество (#)']
unique_norm_abs = norm_abs_df['Норма кол. показателя'].unique()
print("Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Нормобласты, абсолютное количество (#)':")
print(unique_norm_abs)

Уникальные значения в столбце 'Норма кол. показателя' для показателя 'Нормобласты, абсолютное количество (#)':
['_']


In [192]:
unique_ed_norm_abs = norm_abs_df['Ед. изм. кол. показателя'].unique()
print("Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Нормобласты, абсолютное количество (#)':")
print(unique_ed_norm_abs)

Уникальные значения в столбце 'Ед. изм. кол. показателя' для показателя 'Нормобласты, абсолютное количество (#)':
['10*9/л']


In [193]:
unique_znach_norm_abs= norm_abs_df['Значение кол. показателя'].unique()
print("Уникальные значения в столбце 'Значение кол. показателя' для показателя 'Нормобласты, абсолютное количество (#)':")
print(unique_znach_norm_abs)

Уникальные значения в столбце 'Значение кол. показателя' для показателя 'Нормобласты, абсолютное количество (#)':
[0.]


In [194]:
# Подсчет количества записей с 'Кол. лаб. показатель' равным 'Нормобласты, абсолютное количество (#)'
norm_abs_count = df[df['Кол. лаб. показатель'] == 'Нормобласты, абсолютное количество (#)'].shape[0]
norm_abs_count

18

In [195]:
# Удаляем строки, где 'Кол. лаб. показатель' равен 'Миелоциты (лейкоцитарная формула, %)'
df = df[df['Кол. лаб. показатель'] != 'Нормобласты, абсолютное количество (#)']

# Проверяем, что строки успешно удалены
print("Строки с показателем 'Нормобласты, относительное количество (%)' удалены.")

Строки с показателем 'Нормобласты, относительное количество (%)' удалены.


_____________________________________________________________________________________
`Флаг нормы кол. показателя` (тут тоже привести к "Норм", "Пониж", "Повыш"),

In [196]:
# Вывод уникальных значений в столбце 'Флаг нормы кол. показателя'
unique_flags = df['Флаг нормы кол. показателя'].unique()
print("Уникальные значения в столбце 'Флаг нормы кол. показателя':")
print(unique_flags)

Уникальные значения в столбце 'Флаг нормы кол. показателя':
['Пониж' 'Норм' 'Повыш' nan 'Норма']


In [199]:
# Замена значения 'Норма' на 'Норм' в столбце 'Флаг нормы кол. показателя'
df['Флаг нормы кол. показателя'] = df['Флаг нормы кол. показателя'].replace('Норма', 'Норм')

# Проверка результата
print("Обновленные значения в столбце 'Флаг нормы кол. показателя':")
print(df['Флаг нормы кол. показателя'].unique())

Обновленные значения в столбце 'Флаг нормы кол. показателя':
['Пониж' 'Норм' 'Повыш' nan]


In [198]:
# Фильтрация строк, где 'Флаг нормы кол. показателя' равен NaN
nan_rows = df[df['Флаг нормы кол. показателя'].isna()]

# Вывод результата
nan_rows

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Возраст
2378,b36a2106-4308-11ed-ab5a-0050568844e6,Хронический лимфоцитарный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 81х10*9/л,_,1,77
5609,eebeb925-9f8e-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 23*10*9/л,_,1,68
5610,eebeb925-9f8e-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 21,_,1,68
14713,a6cc0de1-ac03-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,шизоциты 4:1000 эритроцитов(№до 10:1000),_,1,66
17789,7b9025cb-ae7c-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты при подсчете по мазку 102,_,0,72
22833,268d2f6a-32ae-11ea-80c5-901b0e633689,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 64*10*9/Л,_,0,86
25849,81f895b9-2ee2-11eb-80e0-901b0e633689,Другой миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,СОЭ - 7 мм/ч,_,1,50
27046,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,Лейкоциты после пересчета (без учета нормоблас...,_,0,61
27047,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,"подсчитано 50 клеток, моноцитоидные элементы -16",_,0,61
27198,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,"моноцитарные элементы с нуклеолами в ядре 63,5",_,0,61


## Что дальше: 

1. Нужно избавиться от пропусков. Посмотреть где они есть, в каких столбцах, узнать причину (может просто в комментарии указана инфа, но не занесена в нужный столбец) и разобраться с этим.

   Больше всего пропусков кстати в столбцах, которые я предлагаю тупо дропнуть (после вытаскивания от туда полезной инфы, конечно, если такая имеется) 

In [50]:
# Проверяем, какие столбцы содержат пропуски и сколько их
missing_info = df.isnull().sum()
print("Количество пропусков по столбцам перед обработкой:")
print(missing_info[missing_info > 0])

Количество пропусков по столбцам перед обработкой:
Кол. лаб. показатель             17
Значение кол. показателя         17
Ед. изм. кол. показателя         17
Норма кол. показателя            17
Флаг нормы кол. показателя       17
Кач. лаб. показатель          28252
Значение кач. показателя      28252
Норма кач. показателя         28252
dtype: int64


In [51]:
# Фильтруем строки с пропусками в столбце "Кол. лаб. показатель" для изучения данных в других столбцах
missing_lab_indicator_df = df[df["Кол. лаб. показатель"].isna()]
missing_lab_indicator_df

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Возраст
2378,b36a2106-4308-11ed-ab5a-0050568844e6,Хронический лимфоцитарный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 81х10*9/л,_,1,77
5609,eebeb925-9f8e-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 23*10*9/л,_,1,68
5610,eebeb925-9f8e-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 21,_,1,68
14713,a6cc0de1-ac03-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,шизоциты 4:1000 эритроцитов(№до 10:1000),_,1,66
17789,7b9025cb-ae7c-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты при подсчете по мазку 102,_,0,72
22833,268d2f6a-32ae-11ea-80c5-901b0e633689,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 64*10*9/Л,_,0,86
25849,81f895b9-2ee2-11eb-80e0-901b0e633689,Другой миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,СОЭ - 7 мм/ч,_,1,50
27046,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,Лейкоциты после пересчета (без учета нормоблас...,_,0,61
27047,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,"подсчитано 50 клеток, моноцитоидные элементы -16",_,0,61
27198,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,"моноцитарные элементы с нуклеолами в ядре 63,5",_,0,61


In [33]:
missing_val_indicator_df = df[df["Норма кол. показателя"].isna()]
missing_val_indicator_df

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Возраст
2378,b36a2106-4308-11ed-ab5a-0050568844e6,Хронический лимфоцитарный лейкоз,Тромбоциты,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 81х10*9/л,_,1,77
5609,eebeb925-9f8e-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Тромбоциты,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 23*10*9/л,_,1,68
5610,eebeb925-9f8e-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Тромбоциты,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 21,_,1,68
14713,a6cc0de1-ac03-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,шизоциты 4:1000 эритроцитов(№до 10:1000),_,1,66
17789,7b9025cb-ae7c-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Тромбоциты,NaN,NaN,NaN,NaN,Комментарий,тромбоциты при подсчете по мазку 102,_,0,72
22833,268d2f6a-32ae-11ea-80c5-901b0e633689,Другой уточненный лейкоз,Тромбоциты,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 64*10*9/Л,_,0,86
25849,81f895b9-2ee2-11eb-80e0-901b0e633689,Другой миелоидный лейкоз,СОЭ,7.0,мм/ч,NaN,NaN,Комментарий,СОЭ - 7 мм/ч,_,1,50
27046,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,Тромбоциты,NaN,NaN,NaN,NaN,Комментарий,Лейкоциты после пересчета (без учета нормоблас...,_,0,61
27047,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,"подсчитано 50 клеток, моноцитоидные элементы -16",_,0,61
27198,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,"моноцитарные элементы с нуклеолами в ядре 63,5",_,0,61
